In [1]:
import pandas as pd
pd.options.display.max_columns = None

import numpy as np
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler

import os
from dotenv import load_dotenv
import boto3
import io
import pymysql
import json

DB Connection

In [ ]:
# from sshtunnel import SSHTunnelForwarder

# class SSHMySQLConnector:
#     def __init__(self):
#         self.ssh_host = None
#         self.ssh_username = None
#         self.ssh_password = None
#         self.db_username = None
#         self.db_password = None
#         self.db_name = None
#         self.tunnel = None
#         self.connection = None

#     def load_config_from_json(self, json_path):
#         try:
#             with open(json_path, 'r', encoding='utf-8') as f:
#                 config = json.load(f)
#                 self.ssh_host = config['ssh_host']
#                 self.ssh_username = config['ssh_username']
#                 self.ssh_password = config['ssh_password']
#                 self.db_username = config['db_username']
#                 self.db_password = config['db_password']
#                 self.db_name = config['db_name']
#         except Exception as e:
#             print("설정 JSON 로딩 실패:", e)

#     def connect(self):
#         try:
#             self.tunnel = SSHTunnelForwarder(
#                 (self.ssh_host, 22),
#                 ssh_username=self.ssh_username,
#                 ssh_password=self.ssh_password,
#                 remote_bind_address=('127.0.0.1', 3306),
#             )
#             self.tunnel.start()
            
#             self.connection = pymysql.connect(
#                 host='127.0.0.1',
#                 port=self.tunnel.local_bind_port,
#                 user=self.db_username,
#                 password=self.db_password,
#                 db=self.db_name,
#                 cursorclass=pymysql.cursors.DictCursor  # 이 줄 추가
#             )
#             print("DB 접속 성공")
#         except Exception as e:
#             print("SSH 또는 DB 연결 실패:", e)

#     def insert_query_with_lookup(self, table_name, data_list):
#         try:
#             with self.connection.cursor() as cursor:
#                 for data in data_list:
#                     # 1. op_member에서 uid, user_id 조회
#                     cursor.execute("""
#                         SELECT uid, user_id, add1_connected FROM op_member
#                         WHERE add1 = %s
#                         LIMIT 1
#                     """, (data['acnt_nm'],))
#                     result = cursor.fetchone()
                    
#                     if result:
#                         data['member_uid'] = result['uid']
#                         data['user_id'] = result['user_id']
#                         data['is_connected'] = result['add1_connected']
#                         # 향후에 ig_user_id가 추가가 된다면, 해당 부분도 확인해서 추가할 수 있게
#                         # data['ig_user_id'] = result['ig_user_id']
#                     else:
#                         data['member_uid'] = 0
#                         data['user_id'] = 'None'
#                         data['is_connected'] = 'n'
#                         # data['ig_user_id'] = 'None'
              

#                     # 2. INSERT 쿼리 구성 및 실행
#                     columns = ', '.join(data.keys())
#                     placeholders = ', '.join([f"%({k})s" for k in data.keys()])
#                     insert_sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
#                     print(insert_sql)
#                     cursor.execute(insert_sql, data)

#                     print(f"inserted acnt_id: {data.get('acnt_id', 'N/A')}")

#             self.S3_S3_CONNection.commit()
#         except Exception as e:
#             self.connection.rollback()
#             print("INSERT 실패:", e)

#     def get_all_infos():
        
#         return 
    
#     def close(self):
#         if self.connection:
#             self.connection.close()
#         if self.tunnel:
#             self.tunnel.stop()

In [3]:
# ssh = SSHMySQLConnector()
# ssh.load_config_from_json('C:/Users/ehddl/Desktop/업무/code/config/ssh_db_config.json')
# ssh.connect()

# data_list = [
#     {
#         'activity_score': 0.18618008163528707,
#         'trend_score': 1.2384506112322682,
#         'follower_total_engagement': 0.9006925508419993,
#         'follower_retention_rate': 4.84593837535013,
#         'avg_post_efficiency': 1.4776093002849746,
#         'acnt_id': '17841400070132367',
#         'acnt_nm': 'gnuoyeatt',
#         'influencer_scale_type': 'nano'
#     }
# ]

# ssh.insert_query_with_lookup('op_mem_seller_score', data_list)


In [4]:
from sshtunnel import SSHTunnelForwarder

class SSHMySQLConnector:
    def __init__(self):
        self.ssh_host = None
        self.ssh_username = None
        self.ssh_password = None
        self.db_username = None
        self.db_password = None
        self.db_name = None
        self.tunnel = None
        self.connection = None

    def load_config_from_json(self, json_path):
        try:
            with open(json_path, 'r', encoding='utf-8') as f:
                config = json.load(f)
                self.ssh_host = config['ssh_host']
                self.ssh_username = config['ssh_username']
                self.ssh_password = config['ssh_password']
                self.db_username = config['db_username']
                self.db_password = config['db_password']
                self.db_name = config['db_name']
        except Exception as e:
            print("설정 JSON 로딩 실패:", e)

    def connect(self, insert=False):
        try:
            self.tunnel = SSHTunnelForwarder(
                (self.ssh_host, 22),
                ssh_username=self.ssh_username,
                ssh_password=self.ssh_password,
                remote_bind_address=('127.0.0.1', 3306),
            )
            self.tunnel.start()
            # insert 여부에 따라 cursorclass 설정
            connect_kwargs = {
                'host': '127.0.0.1',
                'port': self.tunnel.local_bind_port,
                'user': self.db_username,
                'password': self.db_password,
                'db': self.db_name,
            }
            if insert:
                connect_kwargs['cursorclass'] = pymysql.cursors.DictCursor
            self.connection = pymysql.connect(**connect_kwargs)
            print("DB 접속 성공")
        except Exception as e:
            print("SSH 또는 DB 연결 실패:", e)

    def execute_query(self, query):
        # 쿼리 실행 후 데이터를 DataFrame으로 반환
        return pd.read_sql_query(query, self.connection)

    def insert_query_with_lookup(self, table_name, data_list):
        try:
            with self.connection.cursor() as cursor:
                for data in data_list:
                    # 1. op_member에서 uid, user_id 조회
                    cursor.execute("""
                        SELECT uid, user_id, add1_connected FROM op_member
                        WHERE add1 = %s
                        LIMIT 1
                    """, (data['acnt_nm'],))
                    result = cursor.fetchone()
                    
                    if result:
                        data['member_uid'] = result['uid']
                        data['user_id'] = result['user_id']
                        data['is_connected'] = result['add1_connected']
                        # 향후에 ig_user_id가 추가가 된다면, 해당 부분도 확인해서 추가할 수 있게
                        # data['ig_user_id'] = result['ig_user_id']
                    else:
                        data['member_uid'] = 0
                        data['user_id'] = 'None'
                        data['is_connected'] = 'n'
                        # data['ig_user_id'] = 'None'
              

                    # 2. INSERT 쿼리 구성 및 실행
                    columns = ', '.join(data.keys())
                    placeholders = ', '.join([f"%({k})s" for k in data.keys()])
                    insert_sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
                    print(insert_sql)
                    cursor.execute(insert_sql, data)

                    print(f"inserted acnt_id: {data.get('acnt_id', 'N/A')}")

            self.connection.commit()
        except Exception as e:
            self.connection.rollback()
            print("INSERT 실패:", e)
    
    def close(self):
        if self.connection:
            self.connection.close()
        if self.tunnel:
            self.tunnel.stop()

def sendQuery(query):
        ssh = SSHMySQLConnector()
        ssh.load_config_from_json('C:/Users/ehddl/Desktop/업무/code/config/ssh_db_config.json')
        ssh.connect()
        results = ssh.execute_query(query)
        ssh.close()

        return results


In [38]:
def get_all_infos(): 

    query_sales_info = """
        SELECT o.uid, o.add1, s.*
        FROM op_mem_seller_statistics s
        JOIN (
            SELECT member_uid, MAX(regdate) AS max_regdate
            FROM op_mem_seller_statistics
            GROUP BY member_uid
        ) latest ON s.member_uid = latest.member_uid AND s.regdate = latest.max_regdate
        JOIN op_member o ON o.uid = s.member_uid
        JOIN S3_RECENT_USER_INFO_MTR u ON o.add1 = u.acnt_nm
        ORDER BY s.uid DESC
    """
    sales_info = sendQuery(query_sales_info)

    query_seller_interest_info = """
        SELECT
        o.user_id, o.ig_user_id, o.add1, s.interestcategory
        FROM op_member o
        left join op_mem_seller s on o.user_id=s.user_id
        where (o.ig_user_id != '' and o.ig_user_id is not null) or (o.add1 != '' and o.add1 is not null)
    """
    seller_interest_info = sendQuery(query_seller_interest_info)

    query_not_conn_user_main_category_info = """
        SELECT acnt_id, acnt_nm, main_category, top_3_category, is_connected
        FROM INSTAGRAM_USER_CATEGORY_LABELING
    """

    conn_user_main_category_info = sendQuery(query_not_conn_user_main_category_info)

    return sales_info, seller_interest_info, conn_user_main_category_info

In [39]:
sales_info, seller_interest_info, conn_user_main_category_info = get_all_infos()

DB 접속 성공
DB 접속 성공
DB 접속 성공


In [4]:
# ssh = SSHMySQLConnector()
# ssh.load_config_from_json('C:/Users/ehddl/Desktop/업무/code/config/ssh_db_config.json')
# ssh.connect(True)

In [7]:
# data_list = [
#     {
#         'acnt_id': '17841400070132367',
#         'acnt_nm': 'gnuoyeatt',
#         'influencer_scale_type': 'nano'
#     }
# ]

# ssh = SSHMySQLConnector()
# ssh.load_config_from_json('C:/Users/ehddl/Desktop/업무/code/config/ssh_db_config.json')
# ssh.connect(True) # insert 할 때는 True 값을 넣어줘야 함
# ssh.insert_query_with_lookup('op_mem_seller_score', data_list)

Data Loading

In [13]:
# def load_s3_instagram_data():

#     load_dotenv()
#     aws_access_key = os.getenv("aws_accessKey")
#     aws_secret_key = os.getenv("aws_secretKey")
#     region_name='ap-northeast-2'

#     # S3 클라이언트 생성
#     client = boto3.client('s3', aws_access_key_id=aws_access_key, aws_secret_access_key=aws_secret_key, region_name=region_name)

#     today = datetime.now()
#     year, week, _ = today.isocalendar()

#     bucket_name = 'flexmatch-data'

#     table_list = [
#         'CONN_v2_RECENT_USER_INFO_MTR',
#         'CONN_v2_TIME_SERIES_PROFILE_INFO',
#         'CONN_v2_BY_USER_ID_MEDIA_DTL_INFO',
#         'CONN_v2_BY_DATE_MEDIA_AGG_INFO',
#         'CONN_v2_PROFILE_INSIGHT_DTL',
#         'CONN_v2_MEDIA_INSIGHT_CUM',
#         'CONN_v2_PROFILE_INSIGHT_FOLLOWTYPE']
    
#     df_by_table_list = {}

#     for table_name in table_list:
#         prefix = f'instagram-data/tables/{table_name}/2025-07-30/' # 이번주가 없어서 week를 따로 x
#         response = client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

#         if 'Contents' in response:
#             files = [content['Key'] for content in response['Contents'] if content['Key'].endswith('merged_data.parquet')]

#             if not files:
#                 print(f"[Info] No merged_data.parquet found for table: {table_name}")
#                 continue

#             # 각 파일 순회
#             for file_key in files:
#                 try:
#                     obj = client.get_object(Bucket=bucket_name, Key=file_key)
#                     df = pd.read_parquet(io.BytesIO(obj['Body'].read()))
#                     df_by_table_list[table_name] = df
#                     print(f"[Success] Loaded {file_key} for table {table_name}")
#                 except Exception as e:
#                     print(f"[Error] Failed to read {file_key} for table {table_name}: {e}")
#         else:
#             print(f"[Info] No files found under prefix: {prefix}")
        
#     return df_by_table_list

# df_by_table_list = load_s3_instagram_data()

# recent_user_info_mtr_2 = df_by_table_list['CONN_v2_RECENT_USER_INFO_MTR']
# time_series_profile_info_2 = df_by_table_list['CONN_v2_TIME_SERIES_PROFILE_INFO']
# by_user_id_media_dtl_info_2 = df_by_table_list['CONN_v2_BY_USER_ID_MEDIA_DTL_INFO']
# by_date_media_agg_info_2 = df_by_table_list['CONN_v2_BY_DATE_MEDIA_AGG_INFO']
# conn_profile_insight_2 = df_by_table_list['CONN_v2_PROFILE_INSIGHT_DTL']
# conn_media_insight_2 = df_by_table_list['CONN_v2_MEDIA_INSIGHT_CUM']
# conn_profile_insight_followtype_2 = df_by_table_list['CONN_v2_PROFILE_INSIGHT_FOLLOWTYPE']

# recent_user_info_mtr = df_by_table_list['CONN_v2_RECENT_USER_INFO_MTR']
# time_series_profile_info = df_by_table_list['CONN_v2_TIME_SERIES_PROFILE_INFO']
# by_user_id_media_dtl_info = df_by_table_list['CONN_v2_BY_USER_ID_MEDIA_DTL_INFO']
# by_date_media_agg_info = df_by_table_list['CONN_v2_BY_DATE_MEDIA_AGG_INFO']
# conn_profile_insight = df_by_table_list['CONN_v2_PROFILE_INSIGHT_DTL']
# conn_media_insight = df_by_table_list['CONN_v2_MEDIA_INSIGHT_CUM']
# conn_profile_insight_followtype = df_by_table_list['CONN_v2_PROFILE_INSIGHT_FOLLOWTYPE']




In [9]:
# def conn_load_weekly_instagram_data(bucket_name, table_list, target_filename='merged_data.parquet'):
#     # 환경 변수 로딩
#     load_dotenv()
#     aws_access_key = os.getenv("aws_accessKey")
#     aws_secret_key = os.getenv("aws_secretKey")

#     client = boto3.client(
#         's3',
#         aws_access_key_id=aws_access_key,
#         aws_secret_access_key=aws_secret_key,
#         region_name='ap-northeast-2'
#     )

#     today = datetime.now()
#     yesterday = (today - timedelta(days=1))

#     today_date = datetime.now().strftime('%Y-%m-%d')
#     yesterday_date = yesterday.strftime('%Y-%m-%d')

#     # 결과 저장용 딕셔너리 초기화
#     merged_data_by_table = {table_name: {} for table_name in table_list}

#     # 주차별로 데이터 로딩
#     recent_dates = [yesterday_date, today_date]
#     recent_data_by_table = {}

#     for table_name in table_list:
#         recent_data_by_table[table_name] = {}

#         for date_str in recent_dates:
#             prefix = f'instagram-data/tables/{table_name}/{date_str}/'
#             response = client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

#             if 'Contents' not in response:
#                 print(f"[Info] No files found under prefix: {prefix}")
#                 continue

#             target_files = [
#                 content['Key']
#                 for content in response['Contents']
#                 if content['Key'].endswith(target_filename)
#             ]

#             if not target_files:
#                 print(f"[Info] No {target_filename} found for {table_name} on date={date_str}")
#                 continue

#             for file_key in target_files:
#                 try:
#                     obj = client.get_object(Bucket=bucket_name, Key=file_key)
#                     df = pd.read_parquet(io.BytesIO(obj['Body'].read()))
#                     recent_data_by_table[table_name][date_str] = df
#                     print(f"[Success] Loaded {file_key} for table {table_name}, date={date_str}")
#                 except Exception as e:
#                     print(f"[Error] Failed to read {file_key} for {table_name}, date={date_str}: {e}")

#     final_data = {}
#     for table_name, date_data in recent_data_by_table.items():
#         if today_date in date_data and yesterday_date in date_data:
#             final_data[table_name] = {
#                 'yesterday': date_data[yesterday_date],
#                 'today': date_data[today_date]
#             }
#         else:
#             print(f"[Warning] Missing yesterday or today data for table {table_name}")

#     return final_data

In [ ]:
# def load_weekly_instagram_data(bucket_name, table_list, target_filename='merged_data.parquet'):
#     # 환경 변수 로딩
#     load_dotenv()
#     aws_access_key = os.getenv("aws_accessKey")
#     aws_secret_key = os.getenv("aws_secretKey")

#     # S3 클라이언트 초기화
#     client = boto3.client(
#         's3',
#         aws_access_key_id=aws_access_key,
#         aws_secret_access_key=aws_secret_key,
#         region_name='ap-northeast-2'
#     )

#     # 날짜 계산
#     today = datetime.now()
#     day_before_last_week = (today - timedelta(days=7)).strftime('%Y-%m-%d')
#     last_week = (today - timedelta(days=6)).strftime('%Y-%m-%d')
#     recent_dates = [day_before_last_week, last_week]

#     # 테이블별로 결과 저장할 딕셔너리
#     recent_data_by_table = {table_name: {} for table_name in table_list}

#     for table_name in table_list:
#         for date_str in recent_dates:
#             prefix = f'instagram-data/tables/{table_name}/{date_str}/'
#             try:
#                 response = client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

#                 if 'Contents' not in response:
#                     print(f"[Info] No files found under prefix: {prefix}")
#                     continue

#                 target_files = [
#                     content['Key']
#                     for content in response['Contents']
#                     if content['Key'].endswith(target_filename)
#                 ]

#                 if not target_files:
#                     print(f"[Info] No {target_filename} found for {table_name} on {date_str}")
#                     continue

#                 for file_key in target_files:
#                     try:
#                         obj = client.get_object(Bucket=bucket_name, Key=file_key)
#                         df = pd.read_parquet(io.BytesIO(obj['Body'].read()))
#                         recent_data_by_table[table_name][date_str] = df
#                         print(f"[Success] Loaded {file_key} for table {table_name}, date={date_str}")
#                     except Exception as e:
#                         print(f"[Error] Failed to read {file_key} for {table_name} on {date_str}: {e}")

#             except Exception as e:
#                 print(f"[Error] Failed to list objects for {prefix}: {e}")

#     # 결과 구조 정리
#     final_data = {}
#     for table_name, date_data in recent_data_by_table.items():
#         if day_before_last_week in date_data and last_week in date_data:
#             final_data[table_name] = {
#                 'yesterday': date_data[day_before_last_week],
#                 'today': date_data[last_week]
#             }
#         else:
#             print(f"[Warning] Missing data for {table_name} on either {day_before_last_week} or {last_week}")

#     return final_data

In [3]:
# bucket_name = 'flexmatch-data'
# table_list = [
#     'CONN_v2_RECENT_USER_INFO_MTR',
#     'CONN_v2_TIME_SERIES_PROFILE_INFO',
#     'CONN_v2_BY_USER_ID_MEDIA_DTL_INFO',
#     'CONN_v2_BY_DATE_MEDIA_AGG_INFO',
#     'CONN_v2_PROFILE_INSIGHT_DTL',
#     'CONN_v2_MEDIA_INSIGHT_CUM',
#     'CONN_v2_PROFILE_INSIGHT_FOLLOWTYPE'
# ]

# merged_data_by_table = load_weekly_instagram_data(bucket_name, table_list)


In [ ]:
# recent_user_info_mtr = merged_data_by_table['CONN_v2_RECENT_USER_INFO_MTR']['yesterday']
# time_series_profile_info = merged_data_by_table['CONN_v2_TIME_SERIES_PROFILE_INFO']['yesterday']
# by_user_id_media_dtl_info = merged_data_by_table['CONN_v2_BY_USER_ID_MEDIA_DTL_INFO']['yesterday']
# by_date_media_agg_info = merged_data_by_table['CONN_v2_BY_DATE_MEDIA_AGG_INFO']['yesterday']

# recent_user_info_mtr_2 = merged_data_by_table['CONN_v2_RECENT_USER_INFO_MTR']['today']
# time_series_profile_info_2 = merged_data_by_table['CONN_v2_TIME_SERIES_PROFILE_INFO']['today']
# by_user_id_media_dtl_info_2 = merged_data_by_table['CONN_v2_BY_USER_ID_MEDIA_DTL_INFO']['today']
# by_date_media_agg_info_2 = merged_data_by_table['CONN_v2_BY_DATE_MEDIA_AGG_INFO']['today']

# conn_profile_insight = merged_data_by_table['CONN_v2_PROFILE_INSIGHT_DTL']['yesterday']
# conn_profile_insight_2 = merged_data_by_table['CONN_v2_PROFILE_INSIGHT_DTL']['today']

# conn_media_insight = merged_data_by_table['CONN_v2_MEDIA_INSIGHT_CUM']['yesterday']
# conn_media_insight_2 = merged_data_by_table['CONN_v2_MEDIA_INSIGHT_CUM']['today']

# conn_profile_insight_followtype = merged_data_by_table['CONN_v2_PROFILE_INSIGHT_FOLLOWTYPE']['yesterday']
# conn_profile_insight_followtype_2 = merged_data_by_table['CONN_v2_PROFILE_INSIGHT_FOLLOWTYPE']['today']

s3 to db

In [5]:
def get_weekly_instagram_data_from_db(table_list): 
    # db 연결
    ssh = SSHMySQLConnector()
    ssh.load_config_from_json('C:/Users/ehddl/Desktop/업무/code/config/ssh_db_config.json')
    ssh.connect()
    
    # 날짜 계산
    today = datetime.now()
    day_before_last_week = (today - timedelta(days=7)).strftime('%Y-%m-%d')
    last_week = (today - timedelta(days=6)).strftime('%Y-%m-%d')
    recent_dates = [day_before_last_week, last_week]

    recent_data_by_table = {table_name: {} for table_name in table_list}

    for table_name in table_list:
        for date_str in recent_dates:
            try:
                query = f"""
                    SELECT *
                    FROM {table_name}
                    WHERE DATE(reg_date) = '{date_str}'
                """
                df = ssh.execute_query(query)
                if df.empty:
                    print(f"[Info] No data found for {table_name} on {date_str}")
                    continue
                recent_data_by_table[table_name][date_str] = df
                print(f"[Success] Loaded data for {table_name}, date={date_str}")
            except Exception as e:
                print(f"[Error] Failed to query {table_name} on {date_str}: {e}")

    # SSH 연결 종료
    ssh.close()

    # 결과 구조 정리
    final_data = {}
    for table_name, date_data in recent_data_by_table.items():
        if day_before_last_week in date_data and last_week in date_data:
            final_data[table_name] = {
                'yesterday': date_data[day_before_last_week],
                'today': date_data[last_week]
            }
        else:
            print(f"[Warning] Missing data for {table_name} on either {day_before_last_week} or {last_week}")

    return final_data

In [6]:
table_list = [
    # 'S3_CONN_v2_RECENT_USER_INFO_MTR',
    'S3_CONN_v2_TIME_SERIES_PROFILE_INFO',
    'S3_CONN_v2_BY_USER_ID_MEDIA_DTL_INFO',
    'S3_CONN_v2_BY_DATE_MEDIA_AGG_INFO',
    'S3_CONN_v2_PROFILE_INSIGHT_DTL',
    'S3_CONN_v2_MEDIA_INSIGHT_CUM',
    'S3_CONN_v2_PROFILE_INSIGHT_FOLLOWTYPE'
]

merged_data_by_table = get_weekly_instagram_data_from_db(table_list)


DB 접속 성공
[Success] Loaded data for S3_CONN_v2_TIME_SERIES_PROFILE_INFO, date=2025-10-06
[Success] Loaded data for S3_CONN_v2_TIME_SERIES_PROFILE_INFO, date=2025-10-07
[Success] Loaded data for S3_CONN_v2_BY_USER_ID_MEDIA_DTL_INFO, date=2025-10-06
[Success] Loaded data for S3_CONN_v2_BY_USER_ID_MEDIA_DTL_INFO, date=2025-10-07
[Success] Loaded data for S3_CONN_v2_BY_DATE_MEDIA_AGG_INFO, date=2025-10-06
[Success] Loaded data for S3_CONN_v2_BY_DATE_MEDIA_AGG_INFO, date=2025-10-07
[Success] Loaded data for S3_CONN_v2_PROFILE_INSIGHT_DTL, date=2025-10-06
[Success] Loaded data for S3_CONN_v2_PROFILE_INSIGHT_DTL, date=2025-10-07
[Success] Loaded data for S3_CONN_v2_MEDIA_INSIGHT_CUM, date=2025-10-06
[Success] Loaded data for S3_CONN_v2_MEDIA_INSIGHT_CUM, date=2025-10-07
[Success] Loaded data for S3_CONN_v2_PROFILE_INSIGHT_FOLLOWTYPE, date=2025-10-06
[Success] Loaded data for S3_CONN_v2_PROFILE_INSIGHT_FOLLOWTYPE, date=2025-10-07


In [7]:
# recent_user_info_mtr = merged_data_by_table['CONN_v2_RECENT_USER_INFO_MTR']['yesterday']
time_series_profile_info = merged_data_by_table['S3_CONN_v2_TIME_SERIES_PROFILE_INFO']['yesterday']
by_user_id_media_dtl_info = merged_data_by_table['S3_CONN_v2_BY_USER_ID_MEDIA_DTL_INFO']['yesterday']
by_date_media_agg_info = merged_data_by_table['S3_CONN_v2_BY_DATE_MEDIA_AGG_INFO']['yesterday']

# recent_user_info_mtr_2 = merged_data_by_table['S3_CONN_v2_RECENT_USER_INFO_MTR']['today']
time_series_profile_info_2 = merged_data_by_table['S3_CONN_v2_TIME_SERIES_PROFILE_INFO']['today']
by_user_id_media_dtl_info_2 = merged_data_by_table['S3_CONN_v2_BY_USER_ID_MEDIA_DTL_INFO']['today']
by_date_media_agg_info_2 = merged_data_by_table['S3_CONN_v2_BY_DATE_MEDIA_AGG_INFO']['today']

conn_profile_insight = merged_data_by_table['S3_CONN_v2_PROFILE_INSIGHT_DTL']['yesterday']
conn_profile_insight_2 = merged_data_by_table['S3_CONN_v2_PROFILE_INSIGHT_DTL']['today']

conn_media_insight = merged_data_by_table['S3_CONN_v2_MEDIA_INSIGHT_CUM']['yesterday']
conn_media_insight_2 = merged_data_by_table['S3_CONN_v2_MEDIA_INSIGHT_CUM']['today']

conn_profile_insight_followtype = merged_data_by_table['S3_CONN_v2_PROFILE_INSIGHT_FOLLOWTYPE']['yesterday']
conn_profile_insight_followtype_2 = merged_data_by_table['S3_CONN_v2_PROFILE_INSIGHT_FOLLOWTYPE']['today']

In [26]:
by_user_id_media_dtl_info_2

,uid,acnt_id,media_id,media_type_nm,reels_feed_Type_nm,media_url_addr,media_unq_url_addr,tmnl_url_addr,reg_dt,media_cn,acnt_conn_yn,feed_share_yn,cmnt_actvtn_yn,year,week,reg_date
0,38905,17841453615191128,18000404666667486,VIDEO,REELS,https://scontent-nrt1-2.cdninstagram.com/o1/v/...,https://www.instagram.com/reel/DPdJ7i6j8GN/,https://scontent-nrt1-2.cdninstagram.com/v/t51...,2025-10-06 14:21:03,"제품제공 | 타프 메인폴 잘 세우기 [소장각]\n\n홀리데이코리아의 신제품 ""300...",None,1,1,2025,41,2025-10-07 04:04:06
1,38906,17841469251871586,17945226921026005,CAROUSEL_ALBUM,FEED,https://scontent-nrt1-2.cdninstagram.com/v/t51...,https://www.instagram.com/p/DPdZ9t1kl6b/,None,2025-10-06 16:30:28,#협찬 |\n\n동물들의 본성을 존중하는 알모네이처 ( @almonature.kr ...,None,None,1,2025,41,2025-10-07 04:09:04
2,38907,17841400189532541,17988244355894544,CAROUSEL_ALBUM,FEED,https://scontent-nrt1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/DPd3eJkkroJ/,None,2025-10-06 20:48:18,#협찬 @isece_official 쎄쎄(SEECE) 💎✨️\n\n룩을 완성하는 마...,None,None,1,2025,41,2025-10-07 04:15:44
3,38908,17841449994562531,18079915985177776,VIDEO,REELS,https://scontent-nrt1-1.cdninstagram.com/o1/v/...,https://www.instagram.com/reel/DPdWVm5jtAV/,https://scontent-nrt1-1.cdninstagram.com/v/t51...,2025-10-06 16:00:10,추석 OPEN!\n감다살 박보검 한복 화보!\n전세계 보급♡\n\n문화체육관광부 한...,None,1,1,2025,41,2025-10-07 04:15:58
4,38909,17841406217128330,18060140807435473,CAROUSEL_ALBUM,FEED,https://scontent-nrt1-2.cdninstagram.com/v/t51...,https://www.instagram.com/p/DPchf4vAevY/,None,2025-10-06 08:17:03,@studio_holit #광고 \n밝은 미소 담기에 최고인 프로필 전문 서초 홀...,None,None,1,2025,41,2025-10-07 04:21:46


basic preprocessing

In [ ]:
# recent_user_info_mtr = recent_user_info_mtr.drop_duplicates(subset=['acnt_id', 'acnt_nm'])
# time_series_profile_info = time_series_profile_info.drop_duplicates(subset=['base_ymd', 'acnt_id'])
# by_user_id_media_dtl_info = by_user_id_media_dtl_info.drop_duplicates(subset=['acnt_id', 'media_id', 'media_cn'])
# by_date_media_agg_info = by_date_media_agg_info.drop_duplicates(subset=['base_ymd', 'media_id'])

# recent_user_info_mtr_2 = recent_user_info_mtr_2.drop_duplicates(subset=['acnt_id', 'acnt_nm'])
# time_series_profile_info_2 = time_series_profile_info_2.drop_duplicates(subset=['base_ymd', 'acnt_id'])
# by_user_id_media_dtl_info_2 = by_user_id_media_dtl_info_2.drop_duplicates(subset=['acnt_id', 'media_id', 'media_cn'])
# by_date_media_agg_info_2 = by_date_media_agg_info_2.drop_duplicates(subset=['base_ymd', 'media_id'])

# conn_profile_insight = conn_profile_insight.drop_duplicates(subset=['acnt_id', 'acnt_nm'])
# conn_media_insight = conn_media_insight.drop_duplicates(subset=['acnt_id', 'acnt_nm', 'media_id'])

# conn_profile_insight_2 = conn_profile_insight_2.drop_duplicates(subset=['acnt_id', 'acnt_nm'])
# conn_media_insight_2 = conn_media_insight_2.drop_duplicates(subset=['acnt_id', 'acnt_nm', 'media_id'])


가장 최근 데이터로 isin 해줘야 할 것 같음. 그래야 전, 후를 비교할 수 있기 때문임

In [8]:
# 각 acnt_id가 있는 전부 있는 경우만 가능, isin으로 제거
# 근데 전 주차에는 없는 유저가 생길수도 있음
# media 같은 경우에도 최근 100개만 일단 가져오고 있기 때문에 일주일 단위여도 없을수도 있음

# 이렇게 해야하는 이유는 timeseries 변수를 계산하려면 무조건 두 주차에 같은 아이디가 있어야 함
# 또한, user info랑 media info 같이 써야하는 경우에는 어짜피 한번 거르고 있을 듯
# -> 따라서, 굳이 필수적으로 하지는 않아도 될듯함

# unique_user = recent_user_info_mtr['acnt_id'].unique()

# c_unique_user = recent_user_info_mtr_2[recent_user_info_mtr_2['acnt_conn_yn']=='Y']['acnt_id'].to_list()
# c_recent_user_info_mtr_2 = recent_user_info_mtr_2[recent_user_info_mtr_2['acnt_id'].isin(c_unique_user)]

# unique한 user도 timeseries로 대체
all_table_list = [
    time_series_profile_info,
    by_user_id_media_dtl_info,
    by_date_media_agg_info,

    time_series_profile_info_2,
    by_user_id_media_dtl_info_2,
    by_date_media_agg_info_2,

    conn_profile_insight,
    conn_profile_insight_2,

    conn_media_insight,
    conn_media_insight_2,

    conn_profile_insight_followtype,
    conn_profile_insight_followtype_2
    ]

for df in all_table_list:
    if 'acnt_id' in df.columns:
        df['acnt_id'] = df['acnt_id'].astype(str)

c_unique_user = conn_media_insight_2['acnt_id'].to_list()
c_conn_profile_insight_2 = conn_profile_insight_2[conn_profile_insight_2['acnt_id'].isin(c_unique_user)]

# c_time_series_profile_info = time_series_profile_info[time_series_profile_info['acnt_id'].isin(c_unique_user)]
# c_time_series_profile_info_2 = time_series_profile_info_2[time_series_profile_info_2['acnt_id'].isin(c_unique_user)]

# by_user_id_media_dtl_info = by_user_id_media_dtl_info[by_user_id_media_dtl_info['acnt_id'].isin(c_unique_user)]
# c_by_user_id_media_dtl_info_2 = by_user_id_media_dtl_info_2[by_user_id_media_dtl_info_2['acnt_id'].isin(c_unique_user)]

c_time_series_profile_info = time_series_profile_info[time_series_profile_info['acnt_id'].isin(c_unique_user)]
c_time_series_profile_info_2 = time_series_profile_info_2[time_series_profile_info_2['acnt_id'].isin(c_unique_user)]

c_conn_profile_insight_followtype = conn_profile_insight_followtype[conn_profile_insight_followtype['acnt_id'].isin(c_unique_user)]
c_conn_profile_insight_followtype_2 = conn_profile_insight_followtype_2[conn_profile_insight_followtype_2['acnt_id'].isin(c_unique_user)]

# connected_user 같은 경우에는 conn_media_insight 안에 있는 게 media_agg랑 동일하기 때문에 해당 부분을 쓰면 당장은 문제가 없음
# media_dtl_info는 누적이 안되고 있는 문제가 있어서 media_insgith로 대체
unique_media = conn_media_insight_2['media_id'].unique()
# c_by_date_media_agg_info_2 = by_date_media_agg_info_2[by_date_media_agg_info_2['media_id'].isin(unique_media)]
c_conn_media_insight_2 = conn_media_insight_2[conn_media_insight_2['media_id'].isin(unique_media)]

In [20]:
# pd.options.display.max_columns = None
# c_conn_media_insight_2[(c_conn_media_insight_2['acnt_id']=='17841453615191128')&(c_conn_media_insight_2['views_cnt']==0)].sort_values(by=['views_cnt'], ascending=True)

In [9]:
def influencer_scale_type(row):
    count = row['follower_cnt']
    if count < 1000:
        return 'nano'
    elif 1000 <= count <= 10000:
        return 'micro'
    elif 10000 < count <= 100000:
        return 'mid'
    elif 100000 < count <= 500000:
        return 'macro'
    else:
        return 'mega'

# recent_user_info_mtr.loc[:, 'influencer_scale_type'] = recent_user_info_mtr.apply(influencer_scale_type, axis=1)
# c_recent_user_info_mtr_2.loc[:,'influencer_scale_type'] = c_recent_user_info_mtr_2.apply(influencer_scale_type, axis=1)

c_time_series_profile_info_2.loc[:,'influencer_scale_type'] = c_time_series_profile_info_2.apply(influencer_scale_type, axis=1)


create merged file

In [ ]:
# def create_merged_df(user_info_df, timeseries_df, timeseries_df_2, media_info_df, media_insight_df, user_followtype_df, user_followtype_df_2): # media_agg, profile_insight X
#     # merge 시에 같은 이름의 열이 두개여서 error 발생하기 때문에 insight에서는 삭제
#     media_insight = media_insight_df.drop(['acnt_id'], axis=1)
#     media_engagement_merged_df = pd.merge(media_info_df, media_insight, on='media_id', how='outer')
#     # print(len(media_engagement_merged_df['acnt_id'].unique()))

#     ### 방법 1
#     # 단 한개의 게시물이라도 like가 비공개인 influencer 제거 & media_cnt_가 0인 사람도 제외
#     # by_user_na_like_count = media_engagement_merged_df[media_engagement_merged_df['like_cnt'].isna()].groupby(['acnt_id'])['media_id'].count()
#     # na_like_user = by_user_na_like_count[by_user_na_like_count > 0].index
#     # no_media_user = user_info_df[user_info_df['media_cnt'] == 0]['acnt_id'].to_list()
#     # except_user = list(na_like_user) + no_media_user
#     # media_engagement_merged_df = media_engagement_merged_df[~media_engagement_merged_df['acnt_id'].isin(except_user)].reset_index()

#     ### 방법 2
#     # 미디어가 한 개도 없는 유저 제거 & 게시물의 like가 비공개인 경우에는 그걸 제외한 게시물의 좋아요 평균으로 채워넣기
#     # 근데 여기서 like만 비공인건지 나머지 값들도 비공이 되는건지 확인을 하긴 해야함
#     # mean 값 자체가 nan인 사람들도 제외
#     # 근데 여기서 프로페셔널 전환은 이미 되어있지만, views라는 지표 자체가 나중에 나와서 전부 0으로 찍히는 경우가 있음. -> 어떻게 해결?
#     no_media_user = user_info_df[user_info_df['media_cnt'] == 0]['acnt_id'].to_list()
#     media_engagement_merged_df = media_engagement_merged_df[~media_engagement_merged_df['acnt_id'].isin(no_media_user)].reset_index()

#     media_engagement_merged_groupby_df = media_engagement_merged_df.groupby('acnt_id')[['like_cnt', 'cmnt_cnt', 'share_cnt', 'save_cnt', 'views_cnt', 'reach_cnt']].mean()
#     media_engagement_merged_groupby_df = np.ceil(media_engagement_merged_groupby_df)
#     fillna_user = media_engagement_merged_groupby_df[media_engagement_merged_groupby_df['like_cnt'] > 1].index

#     media_engagement_merged_df = media_engagement_merged_df[media_engagement_merged_df['acnt_id'].isin(fillna_user)].reset_index()

#     engagement_cols = ['like_cnt', 'cmnt_cnt', 'share_cnt', 'save_cnt', 'views_cnt', 'reach_cnt']
#     for col in engagement_cols:
#         media_engagement_merged_df[col] = media_engagement_merged_df.apply(
#         lambda row: media_engagement_merged_groupby_df.at[row['acnt_id'], col] if pd.isna(row[col]) else row[col], axis=1)

#     user_list = media_engagement_merged_df['acnt_id'].unique()
#     media_list = media_engagement_merged_df['media_id'].unique()

#     # merge하면서 제거된 리스트가 있기 때문에, 해당 부분 다시 삭제 후에 새로운 merge 파일 생성
#     user_info = user_info_df[user_info_df['acnt_id'].isin(user_list)]
#     timeseries = timeseries_df[timeseries_df['acnt_id'].isin(user_list)]
#     timeseries_2 = timeseries_df_2[timeseries_df_2['acnt_id'].isin(user_list)]
#     media_info = media_info_df[media_info_df['acnt_id'].isin(user_list)]
    
#     user_followtype = user_followtype_df[user_followtype_df['acnt_id'].isin(user_list)]
#     user_followtype_2 = user_followtype_df_2[user_followtype_df_2['acnt_id'].isin(user_list)]
    
#     media_insight_info = media_insight[media_insight['media_id'].isin(media_list)]

#     all_merged_df_a = pd.merge(user_info, timeseries, on='acnt_id')
#     all_merged_df_b = pd.merge(all_merged_df_a, media_info, on='acnt_id')
#     all_merged_df = pd.merge(all_merged_df_b, media_insight_info, on='media_id')
    
#     media_engagement_profile_merged_df = pd.merge(media_engagement_merged_df, user_info_df, on='acnt_id')
#     time_series_merged_df = pd.merge(timeseries, timeseries_df_2, on='acnt_id')

#     return user_info, timeseries, timeseries_2, user_followtype, user_followtype_2, media_info, media_insight_info, all_merged_df, media_engagement_merged_df, media_engagement_profile_merged_df, time_series_merged_df

In [52]:
## DB 변경으로 수정

def create_merged_df(user_profile_insight_df, timeseries_df, timeseries_df_2, media_insight_df, user_followtype_df, user_followtype_df_2): # media_agg, profile_insight X
    # merge 시에 같은 이름의 열이 두개여서 error 발생하기 때문에 insight에서는 삭제
    # media_insight = media_insight_df.drop(['acnt_id'], axis=1)
    # media_insight = pd.merge(media_info_df, media_insight, on='media_id', how='outer')
    # print(len(media_insight['acnt_id'].unique()))

    ### 방법 1
    # 단 한개의 게시물이라도 like가 비공개인 influencer 제거 & media_cnt_가 0인 사람도 제외
    # by_user_na_like_count = media_insight[media_insight['like_cnt'].isna()].groupby(['acnt_id'])['media_id'].count()
    # na_like_user = by_user_na_like_count[by_user_na_like_count > 0].index
    # no_media_user = user_info_df[user_info_df['media_cnt'] == 0]['acnt_id'].to_list()
    # except_user = list(na_like_user) + no_media_user
    # media_insight = media_insight[~media_insight['acnt_id'].isin(except_user)].reset_index()

    ### 방법 2
    # 미디어가 한 개도 없는 유저 제거 & 게시물의 like가 비공개인 경우에는 그걸 제외한 게시물의 좋아요 평균으로 채워넣기
    # 근데 여기서 like만 비공인건지 나머지 값들도 비공이 되는건지 확인을 하긴 해야함
    # mean 값 자체가 nan인 사람들도 제외
    # 근데 여기서 프로페셔널 전환은 이미 되어있지만, views라는 지표 자체가 나중에 나와서 전부 0으로 찍히는 경우가 있음. -> 어떻게 해결?

    # media_cnt => time_series에서 대체
    no_media_user = timeseries_df[timeseries_df['media_cnt'] == 0]['acnt_id'].to_list()
    media_insight = media_insight_df[~media_insight_df['acnt_id'].isin(no_media_user)].reset_index()

    media_engagement_groupby_df = media_insight.groupby('acnt_id')[['like_cnt', 'cmnt_cnt', 'share_cnt', 'save_cnt', 'views_cnt', 'reach_cnt']].mean()
    media_engagement_groupby_df = np.ceil(media_engagement_groupby_df)
    fillna_user = media_engagement_groupby_df[media_engagement_groupby_df['like_cnt'] > 1].index

    media_insight = media_insight[media_insight['acnt_id'].isin(fillna_user)].reset_index()

    engagement_cols = ['like_cnt', 'cmnt_cnt', 'share_cnt', 'save_cnt', 'views_cnt', 'reach_cnt']
    for col in engagement_cols:
        media_insight[col] = media_insight.apply(
        lambda row: media_engagement_groupby_df.at[row['acnt_id'], col] if pd.isna(row[col]) else row[col], axis=1)

    user_list = media_insight['acnt_id'].unique()
    media_list = media_insight['media_id'].unique()

    # merge하면서 제거된 리스트가 있기 때문에, 해당 부분 다시 삭제 후에 새로운 merge 파일 생성
    # unique한 유저도 timeseries에서 대체
    user_info = user_profile_insight_df[user_profile_insight_df['acnt_id'].isin(user_list)]
    timeseries = timeseries_df[timeseries_df['acnt_id'].isin(user_list)]
    timeseries_2 = timeseries_df_2[timeseries_df_2['acnt_id'].isin(user_list)]
    # media_info = media_info_df[media_info_df['acnt_id'].isin(user_list)]
    
    user_followtype = user_followtype_df[user_followtype_df['acnt_id'].isin(user_list)]
    user_followtype_2 = user_followtype_df_2[user_followtype_df_2['acnt_id'].isin(user_list)]
    
    media_insight_info = media_insight[media_insight['media_id'].isin(media_list)]

    all_merged_df_a = pd.merge(user_info, timeseries, on='acnt_id')
    all_merged_df = pd.merge(all_merged_df_a, media_insight_info, on='acnt_id')
    # all_merged_df = pd.merge(all_merged_df_b, media_insight_info, on='media_id')
    
    media_engagement_profile_merged_df = pd.merge(media_insight, user_profile_insight_df, on='acnt_id')
    time_series_merged_df = pd.merge(timeseries, timeseries_df_2, on='acnt_id')
    time_series_merged_df_2 = pd.merge(time_series_merged_df, media_insight_info, on='acnt_id')

    return user_info, timeseries, timeseries_2, user_followtype, user_followtype_2, media_insight_info, all_merged_df, media_engagement_profile_merged_df, time_series_merged_df, time_series_merged_df_2

In [53]:
user_info, timeseries, timeseries_2, user_followtype, user_followtype_2, media_insight_info, all_merged_df, media_engagement_profile_merged_df, time_series_merged_df, time_series_merged_df_2 = create_merged_df(
                                                                                                                        c_conn_profile_insight_2,
                                                                                                                        c_time_series_profile_info,
                                                                                                                        c_time_series_profile_info_2,
                                                                                                                        c_conn_media_insight_2,
                                                                                                                        c_conn_profile_insight_followtype,
                                                                                                                        c_conn_profile_insight_followtype_2)

In [51]:
media_insight_info

,level_0,index,uid,acnt_id,acnt_nm,media_id,acnt_conn_yn,media_type_nm,reels_feed_type_nm,reg_dt,media_cn,like_cnt,share_cnt,views_cnt,save_cnt,cmnt_cnt,reach_cnt,follow_cnt,profile_visit_cnt,profile_activ_cnt,contents_intac_cnt,reels_view_total_time,reels_view_avg_time,base_ymd,year,week,reg_date
0,0,0,69395,17841450980480576,__dongx2_,17997992962899306,Y,CAROUSEL_ALBUM,FEED,2023-06-18 07:20:29,06/18 미금 더볼더\n????\n첫보라 \n스태틱 맛있어\n근데 남색은 리벤지 ...,123,0,0,0,0,60,0,1,0,123,None,None,2025-10-07,2025,41,2025-10-07 04:01:02
1,1,1,69396,17841450980480576,__dongx2_,17998368559882450,Y,CAROUSEL_ALBUM,FEED,2023-05-30 13:25:50,05/30 모란 클라임어스\n????????????????????\n리벤지\n.\n...,161,0,0,0,11,54,0,0,0,172,None,None,2025-10-07,2025,41,2025-10-07 04:01:02
2,2,2,69397,17841450980480576,__dongx2_,18000081436391404,Y,VIDEO,FEED,2021-12-24 08:32:11,???? 이건 도대체 어케하나\n.\n.\n#클린이 #클라이밍 #양재더클라임 #일상...,0,0,0,0,0,0,0,0,0,0,None,None,2025-10-07,2025,41,2025-10-07 04:01:02
3,3,3,69398,17841450980480576,__dongx2_,18001683427825734,Y,CAROUSEL_ALBUM,FEED,2023-08-01 11:50:34,08/01 미금 더볼더\n????????❌️\n하하하 재밌어\n마지막껀 진짜 깨고싶...,144,0,0,1,2,37,0,0,0,147,None,None,2025-10-07,2025,41,2025-10-07 04:01:02
4,4,4,69399,17841450980480576,__dongx2_,18018977155357356,Y,CAROUSEL_ALBUM,FEED,2022-04-14 06:40:29,#36\n????????x9\n클라임 어스 두번째 방문\n평일 낮이라 한시간정도는 ...,0,0,0,0,0,0,0,0,0,0,None,None,2025-10-07,2025,41,2025-10-07 04:01:02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,1075,1180,70680,17841406217128330,ksxxida,17874250824329951,None,CAROUSEL_ALBUM,FEED,2025-09-24 11:47:42,#협찬 #리즈랩 #리즈랩바이알이뮨플러스\n\n국내최초 22종 영양소가 함유된 리즈랩...,34,1,1671,0,0,214,0,39,6,35,None,None,2025-10-07,2025,41,2025-10-07 04:21:49
1076,1076,1181,70681,17841400608483053,danika86_,17855319531531557,None,VIDEO,REELS,2025-09-14 17:48:20,None,4,0,188,0,0,153,0,0,0,4,None,None,2025-10-07,2025,41,2025-10-07 04:21:59
1077,1077,1182,70682,17841445381456278,0o0_kikii,18293305198260700,None,IMAGE,FEED,2025-10-05 00:12:06,#스물둘,42,0,1209,0,4,277,1,20,0,46,None,None,2025-10-07,2025,41,2025-10-07 04:22:08
1078,1078,1183,70683,17841445381456278,0o0_kikii,17985235838896569,None,CAROUSEL_ALBUM,FEED,2025-09-29 02:17:37,#협찬 @standarderror_official \n\n가을 아우터 고민하는 분들...,47,1,2013,0,2,317,0,48,0,50,None,None,2025-10-07,2025,41,2025-10-07 04:22:09


크리에이터 활동성

- 실제 게시물 업로드 날짜를 기준으로 계산할 수 있을 것 같음
- 최근 100개의 게시물의 평균 게시 간격을 계산

In [ ]:
# def calculate_activity_score(recent_media_dtl_df): # 두 개의 테이블 중 가장 최근
#     media_dtl_copy = recent_media_dtl_df.copy()
#     media_dtl_copy = media_dtl_copy.drop_duplicates(subset=['acnt_id', 'media_id', 'media_cn'])
#     media_dtl_copy['reg_dt'] = pd.to_datetime(media_dtl_copy['reg_dt'])
#     media_dtl_copy = media_dtl_copy.sort_values(['acnt_id', 'reg_dt'])

#     # 게시물 간격 계산
#     media_dtl_copy['prev_reg_dt'] = media_dtl_copy.groupby('acnt_id')['reg_dt'].shift(1)
#     media_dtl_copy['gap_days'] = (media_dtl_copy['reg_dt'] - media_dtl_copy['prev_reg_dt']).dt.days

#     # gap_days가 NaN인 첫 번째 포스트 제외 후 평균 간격 계산
#     activity_df = media_dtl_copy.dropna(subset=['gap_days']).groupby('acnt_id')['gap_days'].mean().reset_index()
#     activity_df.rename(columns={'gap_days': 'avg_upload_interval'}, inplace=True)

#     # 활동성 점수 계산 (간격의 역수로 환산) -> 점수 정규화 (업로드 간격이 짧을수록 점수가 높아지도록 역수를 취해서 계산한 것)
#     # 업로드 간격이 너무 짧은 유저의 경우 inf로 계산되는 것을 방지하기 위해서 scaling 진행
#     activity_df['avg_upload_interval'] = activity_df['avg_upload_interval'].replace(0, 0.1)
#     # activity_df['activity_score'] =  activity_df['avg_upload_interval'] / 100
#     activity_df['activity_score'] =  (1 / activity_df['avg_upload_interval']) * 100
    
#     return activity_df

In [12]:
def calculate_activity_score(recent_media_insight_df): # 두 개의 테이블 중 가장 최근
    media_dtl_copy = recent_media_insight_df.copy()
    media_dtl_copy = media_dtl_copy.drop_duplicates(subset=['acnt_id', 'media_id', 'media_cn'])
    media_dtl_copy['reg_dt'] = pd.to_datetime(media_dtl_copy['reg_dt'])
    media_dtl_copy = media_dtl_copy.sort_values(['acnt_id', 'reg_dt'])

    # 게시물 간격 계산
    media_dtl_copy['prev_reg_dt'] = media_dtl_copy.groupby('acnt_id')['reg_dt'].shift(1)
    media_dtl_copy['gap_days'] = (media_dtl_copy['reg_dt'] - media_dtl_copy['prev_reg_dt']).dt.days

    # gap_days가 NaN인 첫 번째 포스트 제외 후 평균 간격 계산
    activity_df = media_dtl_copy.dropna(subset=['gap_days']).groupby('acnt_id')['gap_days'].mean().reset_index()
    activity_df.rename(columns={'gap_days': 'avg_upload_interval'}, inplace=True)

    # 활동성 점수 계산 (간격의 역수로 환산) -> 점수 정규화 (업로드 간격이 짧을수록 점수가 높아지도록 역수를 취해서 계산한 것)
    # 업로드 간격이 너무 짧은 유저의 경우 inf로 계산되는 것을 방지하기 위해서 scaling 진행
    activity_df['avg_upload_interval'] = activity_df['avg_upload_interval'].replace(0, 0.1)
    # activity_df['activity_score'] =  activity_df['avg_upload_interval'] / 100
    activity_df['activity_score'] =  (1 / activity_df['avg_upload_interval']) * 100
    
    return activity_df

In [13]:
activity_df = calculate_activity_score(media_insight_info)

In [14]:
activity_df

,acnt_id,avg_upload_interval,activity_score
0,17841400189532541,0.857143,116.666667
1,17841400539004334,0.050633,1975.000000
2,17841400624415491,21.153846,4.727273
3,17841400703391460,1.000000,100.000000
4,17841401443714725,11.480392,8.710504
5,17841402059097583,1.000000,100.000000
6,17841405735233010,30.312500,3.298969
7,17841406217128330,2.500000,40.000000
8,17841406761867652,1.272727,78.571429
9,17841407411969138,2.000000,50.000000


In [15]:
def check_inf(df):
    float_cols = df.select_dtypes(include=['float64', 'float32']).columns

    mask_inf = np.isinf(df[float_cols]).any(axis=1)
    mask_neginf = np.isneginf(df[float_cols]).any(axis=1)

    invalid_rows = df[mask_inf | mask_neginf]

    print(f"⚠️ inf / -inf 포함 행 개수: {len(invalid_rows)}개")
    display(invalid_rows)

In [16]:
check_inf(activity_df)

⚠️ inf / -inf 포함 행 개수: 0개


,acnt_id,avg_upload_interval,activity_score


트렌드지수 - 팔로워 순 증가율 (follower_growth_rate)

In [ ]:
# time_series_profile_info_2['acnt_id'] = time_series_profile_info_2['acnt_id'].astype(object)

# influencer_list = time_series_profile_info['acnt_id'].unique()
# time_series_profile_info_2[time_series_profile_info_2['acnt_id'].isin(influencer_list)]
# time_series_merged_df = pd.merge(time_series_profile_info, time_series_profile_info_2, on='acnt_id')

# time_series_merged_df['trend_score'] = ((time_series_merged_df['follower_cnt_y'] - time_series_merged_df['follower_cnt_x']) / (time_series_merged_df['follower_cnt_x'])) * 100
# time_series_merged_df[['acnt_id', 'trend_score']]

In [17]:
def calculate_follower_growth_rate(time_series_df, recent_time_series_df):
    time_series_df.loc[:, 'acnt_id'] = time_series_df['acnt_id'].astype(object)
    recent_time_series_df.loc[:, 'acnt_id'] = recent_time_series_df['acnt_id'].astype(object)

    influencer_list = time_series_df['acnt_id'].unique()
    recent_time_series_df = recent_time_series_df[recent_time_series_df['acnt_id'].isin(influencer_list)]
    time_series_merged_df = pd.merge(time_series_df, recent_time_series_df, on='acnt_id')

    time_series_merged_df['follow_growth_rate'] = ((time_series_merged_df['follower_cnt_y'] - time_series_merged_df['follower_cnt_x']) / (time_series_merged_df['follower_cnt_x'])) * 100
    growth_rate_df = time_series_merged_df[['acnt_id', 'follow_growth_rate']]

    return growth_rate_df

In [18]:
growth_rate_df = calculate_follower_growth_rate(timeseries, timeseries_2)

In [19]:
growth_rate_df

,acnt_id,follow_growth_rate
0,17841450980480576,0.000000
1,17841449549923448,-0.026208
2,17841453615191128,0.306583
3,17841402059097583,2.271894
4,17841409045873013,0.000000
5,17841400624415491,0.000000
6,17841405735233010,0.000000
7,17841407411969138,-0.226244
8,17841469251871586,0.382653
9,17841476005109325,0.571429


##### 팔로워 참여도

media_id는 있는데 프로필에 미디어가 없어서 0개인 사람이 있음


In [ ]:
# def calculate_follower_engagement(media_engagement_profile_merged_df):
#     media_engagement_profile_merged_df_copy = media_engagement_profile_merged_df[['acnt_id', 'media_id', 'follower_cnt', 'like_cnt', 'cmnt_cnt', 'share_cnt', 'save_cnt', 'views_cnt', 'reach_cnt', 'media_cnt']]
#     # media_id는 조회가 되지만 실제로 media_cnt는 없는 경우가 있음
#     # media_engagement_profile_merged_df_copy = media_engagement_profile_merged_df_copy[media_engagement_profile_merged_df_copy['media_cnt'] != 0]
    
#     engaged_df = media_engagement_profile_merged_df_copy.groupby(['acnt_id']).agg({
#         'like_cnt' : 'sum',
#         'cmnt_cnt' : 'sum',
#         'share_cnt' : 'sum',
#         'save_cnt' : 'sum',
#         'media_id' : 'count',
#         'follower_cnt' : 'first',
#     }).reset_index()

#     engaged_df['estimated_total_engagement'] = ((engaged_df['like_cnt'] + engaged_df['cmnt_cnt'] + engaged_df['share_cnt'] + engaged_df['save_cnt']) / ( engaged_df['media_id']*engaged_df['follower_cnt']))
#     engaged_df['follower_total_engagement'] = engaged_df['estimated_total_engagement'] * 100
    
#     follower_engagment_df = engaged_df

#     return follower_engagment_df

In [ ]:
# follower_engagment_df = calculate_follower_engagement(media_engagement_profile_merged_df)
# follower_engagment_df

In [ ]:
# check_inf(follower_engagment_df)

##### 팔로워 충성도

- 비즈니스 계정 연동 시 follower_type 데이터 로딩 -> 실제 follower cnt랑 안맞음 (timeseries랑 안맞음)

In [ ]:
# def calculate_follower_loyalty(time_series_merged_df):
#     time_series_merged_df_copy = time_series_merged_df[['acnt_id', 'follower_cnt_x', 'follower_cnt_y']].copy()

#     time_series_merged_df_copy.loc[:, 'follower_change'] = (time_series_merged_df_copy['follower_cnt_y'] - time_series_merged_df_copy['follower_cnt_x'])

#     def estimate_new_follower(row):
#         if row['follower_change'] < 0:
#             return 0
#         else:
#             return row['follower_change']

#     time_series_merged_df_copy.loc[:, 'new_follower'] = time_series_merged_df_copy.apply(estimate_new_follower, axis=1)
#     time_series_merged_df_copy.loc[:, 'unfollowed'] = time_series_merged_df_copy['follower_cnt_x'] + time_series_merged_df_copy['new_follower'] - time_series_merged_df_copy['follower_cnt_y']
#     time_series_merged_df_copy.loc[:, 'follower_retention_rate'] = ((time_series_merged_df_copy['follower_cnt_x'] - time_series_merged_df_copy['unfollowed']) / time_series_merged_df_copy['follower_cnt_x']) * 100
#     time_series_merged_df_copy.loc[:, 'follower_retention_rate'] = time_series_merged_df_copy['follower_retention_rate'].round(2)

#     follower_loyalty_df = time_series_merged_df_copy

#     return follower_loyalty_df

In [22]:
# follower_loyalty_df = calculate_follower_loyalty(time_series_merged_df)
# follower_loyalty_df

,acnt_id,follower_cnt_x,follower_cnt_y,follower_change,new_follower,unfollowed,follower_retention_rate
0,17841409045873013,108,108,0,0,0,100.00
1,17841400360358101,127,127,0,0,0,100.00
2,17841449549923448,49957,49954,-3,0,3,99.99
3,17841453615191128,37247,37837,590,590,0,100.00
4,17841400624415491,439,439,0,0,0,100.00
5,17841472921767324,321,320,-1,0,1,99.69
6,17841401204926746,553,552,-1,0,1,99.82
7,17841467094117763,40810,40791,-19,0,19,99.95
8,17841405735233010,607,608,1,1,0,100.00
9,17841400539004334,91027,91016,-11,0,11,99.99


In [23]:
check_inf(follower_loyalty_df)

⚠️ inf / -inf 포함 행 개수: 0개


,acnt_id,follower_cnt_x,follower_cnt_y,follower_change,new_follower,unfollowed,follower_retention_rate


In [24]:
# follow_type_df = conn_profile_insight_followtype_2.pivot_table(
#     index=["acnt_id", "base_ymd", "acnt_nm"],  # 고유 식별자 기준으로 고정
#     columns="follow_type_nm",  # FOLLOWER / NON_FOLLOWER가 컬럼으로 올라감
#     values="follow_unfollow_cnt",  # 우리가 관심있는 값
#     fill_value=0  # 값이 없을 경우 0으로 채움
# )

# # 컬럼명 정리
# follow_type_df.columns = [f"{col.lower()}_follow_unfollow_cnt" for col in follow_type_df.columns]
# follow_type_df.rename(columns={'follower_follow_unfollow_cnt':'new_follower', 'non_follower_follow_unfollow_cnt':'unfollowed'}, inplace=True)
# follow_type_df = follow_type_df.reset_index()
# follow_type_df

,acnt_id,base_ymd,acnt_nm,new_follower,unfollowed
0,17841400337192059,2025-08-14,left_sson,2.0,0.0
1,17841400360358101,2025-08-14,bro_bums,1.0,0.0
2,17841400539004334,2025-08-14,bbangsori,14.0,41.0
3,17841400624415491,2025-08-14,itsme_ddonni,1.0,2.0
4,17841401204926746,2025-08-14,00._.nnny,1.0,0.0
5,17841401246114643,2025-08-14,___eunoia_b,0.0,0.0
6,17841405735233010,2025-08-14,flex_h.y,0.0,0.0
7,17841408178762264,2025-08-14,pwangjunn,0.0,0.0
8,17841409045873013,2025-08-14,__cherie_ssom,0.0,0.0
9,17841449549923448,2025-08-14,heihwi,51.0,20.0


In [ ]:
# timeseries_2_copy = timeseries_2[['acnt_id', 'follower_cnt']]
# # time_series_merged_df = time_series_merged_df[['acnt_id', 'follower_cnt_x', 'base_ymd_x', 'follower_cnt_y', 'base_ymd_y']]
# follow_type_df = pd.merge(follow_type_df, timeseries_2_copy, on='acnt_id', how='left')
# follow_type_df_copy = follow_type_df.copy()
# follow_type_df_copy.dropna(inplace=True)

In [ ]:
# # # 결국엔 기존에 계산하던 방식과 동일한 셈. 다만, 기존에는 신규가입자가 없다는 가정하에 줄어들면 전부 언팔로 간주한거라서 약간의 차이 정도만 있음
# follow_type_df_copy['unfollow_rate'] = (follow_type_df_copy['unfollowed'] / follow_type_df_copy['follower_cnt']) * 100
# # follow_type_df_copy['follower_loyalty_score_1'] = 100 - follow_type_df_copy['unfollow_rate']

In [ ]:
# # 전체 규모 대비 실제 순 증감이 어느정도 인가?
# # 더 큰 채널일수록 들어오는 팔로워도 많지만 이탈도 많을 수 있으므로 밸런스를 봄
# follow_type_df_copy['net_gain'] = follow_type_df_copy['new_follower'] - follow_type_df_copy['unfollowed']
# follow_type_df_copy['follower_loyalty_score_2'] = (follow_type_df_copy['net_gain'] / (follow_type_df_copy['follower_cnt'] + follow_type_df_copy['new_follower'])) * 100

In [ ]:
# # 팔로워의 +-를 전부 예상해서 계산한 값
# follow_type_df_copy.loc[:, 'follower_loyalty_score_3'] = ((follow_type_df_copy['follower_cnt'] - follow_type_df_copy['net_gain']) / follow_type_df_copy['follower_cnt']) * 100

In [ ]:
# follow_type_df_copy['retained_follower'] = follow_type_df_copy['follower_cnt'] - follow_type_df_copy['unfollowed']
# follow_type_df_copy['follower_retention_rate'] = (follow_type_df_copy['retained_follower'] / follow_type_df_copy['follower_cnt']) * 100

# 이거 같은 경우에는 사실 follower_cnt 자체가 unfollowe가 이미 반영된 상태

In [ ]:
# follow_type_df_copy['churn_ratio'] = (follow_type_df_copy['unfollowed'] / (follow_type_df_copy['new_follower'] + 1e-5)).round(2)
# follow_type_df_copy['follower_loyalty_score_3'] = pd.cut(follow_type_df_copy['churn_ratio'], bins=[-0.1, 0.5, 1.0, 2.0, float('inf')], labels=["High", "Moderate", "Low", "Critical"])

In [20]:
def calculate_follower_loyalty(conn_profile_insight_followtype_2, timeseries_2):
    follow_type_df = conn_profile_insight_followtype_2.pivot_table(
        index=["acnt_id", "base_ymd", "acnt_nm"],  # 고유 식별자 기준으로 고정
        columns="follow_type_nm",  # FOLLOWER / NON_FOLLOWER가 컬럼으로 올라감
        values="follow_unfollow_cnt",  # 우리가 관심있는 값
        fill_value=0  # 값이 없을 경우 0으로 채움
    )

    # preprocessing follow_type_df
    follow_type_df.columns = [f"{col.lower()}_follow_unfollow_cnt" for col in follow_type_df.columns]
    follow_type_df.rename(columns={'follower_follow_unfollow_cnt':'new_follower', 'non_follower_follow_unfollow_cnt':'unfollowed'}, inplace=True)
    follow_type_df = follow_type_df.reset_index()
    follow_type_df

    timeseries_2_copy = timeseries_2[['acnt_id', 'follower_cnt']]
    follow_type_df_copy = follow_type_df.copy()
    follow_type_df_copy = pd.merge(follow_type_df, timeseries_2_copy, on='acnt_id', how='left')
    follow_type_df_copy.dropna(inplace=True)

    follow_type_df_copy['unfollow_rate'] = (follow_type_df_copy['unfollowed'] / follow_type_df_copy['follower_cnt']) * 100
    follow_type_df_copy['net_gain'] = follow_type_df_copy['new_follower'] - follow_type_df_copy['unfollowed']
    follow_type_df_copy['follower_retention_rate'] = (follow_type_df_copy['net_gain'] / (follow_type_df_copy['follower_cnt'])) * 100

    return follow_type_df_copy

In [21]:
follower_loyalty_df = calculate_follower_loyalty(conn_profile_insight_followtype_2, timeseries_2)
follower_loyalty_df

,acnt_id,base_ymd,acnt_nm,new_follower,unfollowed,follower_cnt,unfollow_rate,net_gain,follower_retention_rate
0,17841400189532541,2025-10-07,24oldfency,8.0,13.0,33267.0,0.039078,-5.0,-0.015030
1,17841400539004334,2025-10-07,bbangsori,12.0,12.0,90677.0,0.013234,0.0,0.000000
2,17841400608483053,2025-10-07,danika86_,0.0,2.0,10244.0,0.019524,-2.0,-0.019524
3,17841400624415491,2025-10-07,itsme_ddonni,0.0,0.0,434.0,0.000000,0.0,0.000000
4,17841400703391460,2025-10-07,oil.hwa,1.0,1.0,1415.0,0.070671,0.0,0.000000
5,17841401443714725,2025-10-07,daily56789,1.0,2.0,1425.0,0.140351,-1.0,-0.070175
6,17841402059097583,2025-10-07,zenyaluiah,0.0,1.0,5582.0,0.017915,-1.0,-0.017915
8,17841404788541667,2025-10-07,haneum0623,1.0,1.0,24127.0,0.004145,0.0,0.000000
9,17841405735233010,2025-10-07,flex_h.y,1.0,0.0,601.0,0.000000,1.0,0.166389
10,17841406217128330,2025-10-07,ksxxida,2.0,2.0,4786.0,0.041789,0.0,0.000000


콘텐츠 효율성 평가

- 콘텐츠 1개당 팔로워 대비 반응량(like+comments+share+save)
- media_agg & media_dtl & acnt_id가 필요 -> 전부 병합한 게 media_engagement_merged_df
- 단, 프로페셔널 전환은 이전에 했으나, views라는 지표가 나중에 나와서 0으로 찍히는 애들이 있음 -> 이런 부분은 전부 inf로 계산이 되는데 전부 제거하고 mean을 구하는 것으로 !


In [25]:
def calculate_post_efficiency_df(media_engagement_profile_merged_df):
    media_engagement_profile_merged_df_copy = media_engagement_profile_merged_df.copy()

    media_engagement_profile_merged_df_copy['post_efficiency'] = ((media_engagement_profile_merged_df_copy['like_cnt_x'] + media_engagement_profile_merged_df_copy['cmnt_cnt_x'] + media_engagement_profile_merged_df_copy['save_cnt_x'] + media_engagement_profile_merged_df_copy['share_cnt_x']) / media_engagement_profile_merged_df_copy['follower_cnt']) * 100
    post_efficiency_df = media_engagement_profile_merged_df_copy.groupby('acnt_id')['post_efficiency'].mean().reset_index()
    post_efficiency_df.rename(columns={'post_efficiency': 'avg_post_efficiency'}, inplace=True)

    return post_efficiency_df

In [30]:
post_efficiency_df = calculate_post_efficiency_df(all_merged_df)
post_efficiency_df

,acnt_id,avg_post_efficiency
0,17841400189532541,0.817603
1,17841400539004334,7.031801
2,17841400608483053,0.000000
3,17841400624415491,0.000000
4,17841400703391460,0.070621
5,17841401443714725,0.140252
6,17841402059097583,0.036643
7,17841404788541667,0.000000
8,17841405735233010,0.000000
9,17841406217128330,2.193899


콘텐츠 인기도 평가

- 콘텐츠 1개당 조회수 대비 반응량(like+comments+share+save)
- media_agg & media_dtl & acnt_id가 필요 -> 전부 병합한 게 media_engagement_merged_df
- 단, 프로페셔널 전환은 이전에 했으나, views라는 지표가 나중에 나와서 0으로 찍히는 애들이 있음 -> 이런 부분은 전부 inf로 계산이 되는데 전부 제거하고 mean을 구하는 것으로 !

In [33]:
def calculate_post_popularity_df(media_engagement_profile_merged_df):
    media_engagement_profile_merged_df_copy = media_engagement_profile_merged_df.copy()
    media_engagement_profile_merged_df_copy['views_cnt'] = media_engagement_profile_merged_df_copy['views_cnt_x'].replace(0, 1)
    media_engagement_profile_merged_df_copy['post_popularity'] = ((media_engagement_profile_merged_df_copy['like_cnt_x'] + media_engagement_profile_merged_df_copy['cmnt_cnt_x'] + media_engagement_profile_merged_df_copy['save_cnt_x'] + media_engagement_profile_merged_df_copy['share_cnt_x']) / media_engagement_profile_merged_df_copy['views_cnt_x']) * 100
    # check_inf(media_engagement_profile_merged_df_copy)
    media_engagement_profile_merged_df_copy.replace([np.inf, -np.inf], np.nan, inplace=True)
    # media_engagement_profile_merged_df_copy.dropna(subset=['post_popularity'], inplace=True)

    post_popularity_df = media_engagement_profile_merged_df_copy.groupby('acnt_id')['post_popularity'].mean().reset_index()
    post_popularity_df.rename(columns={'post_popularity': 'avg_post_popularity'}, inplace=True)

    return post_popularity_df

In [34]:
post_popularity_df = calculate_post_popularity_df(media_engagement_profile_merged_df)
post_popularity_df

,acnt_id,avg_post_popularity
0,17841400189532541,5.606877
1,17841400539004334,0.566275
2,17841400608483053,2.127660
3,17841400624415491,2.528224
4,17841400703391460,3.084597
5,17841401443714725,9.463021
6,17841402059097583,1.774760
7,17841404788541667,67.940552
8,17841405735233010,NaN
9,17841406217128330,4.544899


광고효율성

In [40]:
def calculate_ad_efficiency(conn_user_main_category_info, sales_info):
    conn_user_main_category_info['add1'] = conn_user_main_category_info['acnt_nm']
    db_merged_data = pd.merge(sales_info, conn_user_main_category_info, on='add1')
    db_merged_data = db_merged_data[['uid', 'user_id', 'member_uid', 'add1', 'acnt_id', 'acnt_nm', 'storeid', 'total_visit', 'total_order', 'match_total_price']]

    db_merged_data['acnt_id'] = db_merged_data['acnt_id'].astype(str)
    db_merged_data_2 = pd.merge(db_merged_data, post_efficiency_df, on='acnt_id', how='left')

    db_merged_data_3= db_merged_data_2[(db_merged_data_2['total_order']!=0) & (db_merged_data_2['match_total_price']!=0)].dropna()
    db_merged_data_3['ad_efficiency'] = db_merged_data_3['total_order'] / db_merged_data_3['avg_post_efficiency']
    
    return db_merged_data_3

In [41]:
db_merged_data_3 = calculate_ad_efficiency(conn_user_main_category_info, sales_info)

In [42]:
db_merged_data_3

,uid,uid,user_id,member_uid,add1,acnt_id,acnt_nm,storeid,total_visit,total_order,match_total_price,avg_post_efficiency,ad_efficiency
0,5196,437159,samelee48,5196,__dongx2_,17841450980480576,__dongx2_,samelee48,515,20,1566750,0.000000,inf
19,6997,436488,ididas,6997,bong_camper83,17841453615191128,bong_camper83,ididas,451,114,6832200,7.732453,14.743057


##### connected_influencer_flexmatch_score

In [ ]:
# def connected_user_flexmatch_score(activity_df, growth_rate_df, follower_engagement_df, follower_loyalty_df, post_efficiency_df, post_popularity_df):
#     # 크리에이터 활동성
#     creator_activity_score = activity_df[['acnt_id', 'activity_score']]
#     # 트렌드지수 (팔로워 순변화량)
#     creator_follow_growth_rate = growth_rate_df[['acnt_id', 'follow_growth_rate']] # db 변수명 수정
#     # 팔로워 참여도
#     follower_engagement = follower_engagement_df[['acnt_id', 'follower_total_engagement']]
#     # 팔로워 충성도
#     follower_loyalty = follower_loyalty_df[['acnt_id', 'follower_retention_rate']]
#     # 콘텐츠 효율성
#     post_efficiency = post_efficiency_df[['acnt_id', 'avg_post_efficiency']]
#     # 콘텐츠 인기도
#     post_popularity = post_popularity_df[['acnt_id', 'avg_post_popularity']]


#     # data_list
#     df_list = [creator_activity_score, creator_follow_growth_rate, follower_engagement, follower_loyalty, post_efficiency, post_popularity]

#     from functools import reduce

#     flexmatch_score = reduce(lambda left, right: pd.merge(left, right, on='acnt_id', how='left'), df_list)
#     user_info_nm = user_info[['acnt_id', 'acnt_nm', 'influencer_scale_type']]
#     flexmatch_score = pd.merge(flexmatch_score, user_info_nm, on='acnt_id')
#     flexmatch_score = flexmatch_score[['acnt_id', 'acnt_nm', 'influencer_scale_type', 'activity_score', 'follow_growth_rate', 'follower_total_engagement', 'follower_retention_rate', 'avg_post_efficiency', 'avg_post_popularity']]

#     connected_flexmatch_score_table = flexmatch_score.copy()
#     connected_flexmatch_score_table.dropna(inplace=True)
    
#     return connected_flexmatch_score_table

In [ ]:
# def connected_user_flexmatch_score(activity_df, growth_rate_df, follower_loyalty_df, post_efficiency_df, post_popularity_df, ad_efficiency_df):
#     # 크리에이터 활동성
#     creator_activity_score = activity_df[['acnt_id', 'activity_score']]
#     # 트렌드지수 (팔로워 순변화량)
#     creator_follow_growth_rate = growth_rate_df[['acnt_id', 'follow_growth_rate']] # db 변수명 수정
#     # 팔로워 참여도
#     # follower_engagement = follower_engagement_df[['acnt_id', 'follower_total_engagement']]
#     # 팔로워 충성도
#     follower_loyalty = follower_loyalty_df[['acnt_id', 'follower_retention_rate']]
#     # 콘텐츠 효율성
#     post_efficiency = post_efficiency_df[['acnt_id', 'avg_post_efficiency']]
#     # 콘텐츠 인기도
#     post_popularity = post_popularity_df[['acnt_id', 'avg_post_popularity']]
#     # 광고 효율성
#     ad_efficiency = ad_efficiency_df[['acnt_id', 'ad_efficiency']]


#     # data_list
#     df_list = [creator_activity_score, creator_follow_growth_rate, follower_loyalty, post_efficiency, post_popularity, ad_efficiency]

#     from functools import reduce

#     flexmatch_score = reduce(lambda left, right: pd.merge(left, right, on='acnt_id', how='outer'), df_list)
#     flexmatch_score['ad_efficiency'] = flexmatch_score['ad_efficiency'].fillna(0)
    
#     user_info_nm = user_info[['acnt_id', 'acnt_nm', 'influencer_scale_type']]
#     flexmatch_score = pd.merge(flexmatch_score, user_info_nm, on='acnt_id')
#     flexmatch_score = flexmatch_score[['acnt_id', 'acnt_nm', 'influencer_scale_type', 'activity_score', 'follow_growth_rate', 'follower_retention_rate', 'avg_post_efficiency', 'avg_post_popularity', 'ad_efficiency']]

#     connected_flexmatch_score_table = flexmatch_score.copy()
#     # connected_flexmatch_score_table.dropna(inplace=True)
    
#     return connected_flexmatch_score_table

In [73]:
def connected_user_flexmatch_score(activity_df, growth_rate_df, follower_loyalty_df, post_efficiency_df, post_popularity_df, ad_efficiency_df):
    # 크리에이터 활동성
    creator_activity_score = activity_df[['acnt_id', 'activity_score']]
    # 트렌드지수 (팔로워 순변화량)
    creator_follow_growth_rate = growth_rate_df[['acnt_id', 'follow_growth_rate']] # db 변수명 수정
    # 팔로워 참여도
    # follower_engagement = follower_engagement_df[['acnt_id', 'follower_total_engagement']]
    # 팔로워 충성도
    follower_loyalty = follower_loyalty_df[['acnt_id', 'follower_retention_rate']]
    # 콘텐츠 효율성
    post_efficiency = post_efficiency_df[['acnt_id', 'avg_post_efficiency']]
    # 콘텐츠 인기도
    post_popularity = post_popularity_df[['acnt_id', 'avg_post_popularity']]
    # 광고 효율성
    ad_efficiency = ad_efficiency_df[['acnt_id', 'ad_efficiency']]


    # data_list
    df_list = [creator_activity_score, creator_follow_growth_rate, follower_loyalty, post_efficiency, post_popularity, ad_efficiency]

    from functools import reduce

    flexmatch_score = reduce(lambda left, right: pd.merge(left, right, on='acnt_id', how='outer'), df_list)
    flexmatch_score['ad_efficiency'] = flexmatch_score['ad_efficiency'].fillna(0)
    
    user_info_nm = time_series_merged_df_2[['acnt_id', 'acnt_nm', 'influencer_scale_type']]
    flexmatch_score = pd.merge(flexmatch_score, user_info_nm, on='acnt_id', how='inner')
    flexmatch_score = flexmatch_score[['acnt_id', 'acnt_nm', 'influencer_scale_type', 'activity_score', 'follow_growth_rate', 'follower_retention_rate', 'avg_post_efficiency', 'avg_post_popularity', 'ad_efficiency']]

    connected_flexmatch_score_table = flexmatch_score.copy()
    connected_flexmatch_score_table = connected_flexmatch_score_table.drop_duplicates(subset = ['acnt_id', 'acnt_nm']).reset_index(drop=True)
    # connected_flexmatch_score_table.dropna(inplace=True)
    
    return connected_flexmatch_score_table

In [74]:
connected_flexmatch_score_table = connected_user_flexmatch_score(activity_df, growth_rate_df, follower_loyalty_df, post_efficiency_df, post_popularity_df, db_merged_data_3)
connected_flexmatch_score_table

,acnt_id,acnt_nm,influencer_scale_type,activity_score,follow_growth_rate,follower_retention_rate,avg_post_efficiency,avg_post_popularity,ad_efficiency
0,17841400189532541,24oldfency,mid,116.666667,-0.003006,-0.015030,0.817603,5.606877,0.000000
1,17841400539004334,bbangsori,mid,1975.000000,-0.012129,0.000000,7.031801,0.566275,0.000000
2,17841400608483053,danika86_,mid,NaN,0.000000,-0.019524,0.000000,2.127660,0.000000
3,17841400624415491,itsme_ddonni,nano,4.727273,0.000000,0.000000,0.000000,2.528224,0.000000
4,17841400703391460,oil.hwa,micro,100.000000,-0.070621,0.000000,0.070621,3.084597,0.000000
5,17841401443714725,daily56789,micro,8.710504,-0.070126,-0.070175,0.140252,9.463021,0.000000
6,17841402059097583,zenyaluiah,micro,100.000000,2.271894,-0.017915,0.036643,1.774760,0.000000
7,17841404788541667,haneum0623,mid,NaN,0.000000,0.000000,0.000000,67.940552,0.000000
8,17841405735233010,flex_h.y,nano,3.298969,0.000000,0.166389,0.000000,NaN,0.000000
9,17841406217128330,ksxxida,micro,40.000000,0.000000,0.000000,2.193899,4.544899,0.000000


In [75]:
seller_interest_info['ig_user_id'] = seller_interest_info['ig_user_id'].replace('', np.nan, regex=True)
conn_user = seller_interest_info[(seller_interest_info['ig_user_id'].notnull()) & (seller_interest_info['ig_user_id'] != '')]
conn_user_interestcategory = conn_user[['ig_user_id', 'interestcategory']].rename({'ig_user_id' : 'acnt_id'}, axis=1)
connected_flexmatch_score_table = pd.merge(connected_flexmatch_score_table, conn_user_interestcategory, on='acnt_id')

In [76]:
conn_user_main_category_info = conn_user_main_category_info[['acnt_id', 'main_category', 'top_3_category']]
conn_user_main_category_info['acnt_id'] = conn_user_main_category_info['acnt_id'].astype(str)

connected_flexmatch_score_table = pd.merge(connected_flexmatch_score_table, conn_user_main_category_info, on='acnt_id', how='inner')

In [77]:
connected_flexmatch_score_table

,acnt_id,acnt_nm,influencer_scale_type,activity_score,follow_growth_rate,follower_retention_rate,avg_post_efficiency,avg_post_popularity,ad_efficiency,interestcategory,main_category,top_3_category
0,17841400189532541,24oldfency,mid,116.666667,-0.003006,-0.015030,0.817603,5.606877,0.000000,None,뷰티,뷰티@유명장소/핫플@여행/관광
1,17841400539004334,bbangsori,mid,1975.000000,-0.012129,0.000000,7.031801,0.566275,0.000000,패션,푸드,푸드@유명장소/핫플@뷰티
2,17841402059097583,zenyaluiah,micro,100.000000,2.271894,-0.017915,0.036643,1.774760,0.000000,패션@뷰티@홈/리빙@푸드,뷰티,뷰티@다이어트/건강보조식품@패션
3,17841405735233010,flex_h.y,nano,3.298969,0.000000,0.166389,0.000000,NaN,0.000000,홈/리빙,일상,일상@반려동물
4,17841449549923448,heihwi,mid,34.455959,-0.026208,-0.020165,0.334650,2.901740,0.000000,패션,다이어트/건강보조식품,다이어트/건강보조식품@패션@일상
5,17841449994562531,twinkle_star_0710,nano,15.384615,0.000000,0.000000,35.135135,3.549261,0.000000,뷰티@푸드@스포츠,엔터테인먼트,엔터테인먼트@유명장소/핫플@여행/관광
6,17841450980480576,__dongx2_,nano,8.284519,0.000000,0.141443,0.000000,13.043271,inf,베이비/키즈,스포츠,스포츠@일상@다이어트/건강보조식품
7,17841453615191128,bong_camper83,mid,101.910828,0.306583,0.536631,7.732453,2.831745,14.743057,홈/리빙@푸드@베이비/키즈,홈/리빙,홈/리빙@유명장소/핫플@여행/관광
8,17841467094117763,hooyexni,mid,16.590909,-0.040389,-0.040405,0.055535,3.931498,0.000000,패션@뷰티@스포츠,일상,일상@여행/관광@뷰티


In [96]:
# connected_flexmatch_score_table['main_interest_category'] = connected_flexmatch_score_table['interestcategory'].str.split('@').str[0]
# connected_flexmatch_score_table

In [75]:
# def normalize_influencer_scores(influencer_scale_names, influencer_scale_df_list, reverse_columns=None, feature_range=(0, 5)):

#     if reverse_columns is None:
#         reverse_columns = ['activity_score']

#     normalized_df_dict = {}

#     for name, df in zip(influencer_scale_names, influencer_scale_df_list):
#         cleaned = df.copy()

#         # 무한대 및 NaN 제거
#         float_cols = cleaned.select_dtypes(include='float64').columns
#         cleaned[float_cols] = cleaned[float_cols].replace([np.inf, -np.inf], np.nan)
#         cleaned = cleaned.dropna(subset=float_cols)

#         if cleaned.empty:
#             continue

#         norm_df = pd.DataFrame(index=cleaned.index)
#         for col in float_cols:

#             scaler = MinMaxScaler(feature_range=feature_range)
#             norm_col = scaler.fit_transform(cleaned[[col]])
#             if col in reverse_columns:
#                 norm_df[col] = feature_range[1] - norm_col.ravel()
#             else:
#                 norm_df[col] = norm_col.ravel()

#         # ID 및 이름, 스케일 타입 추가
#         norm_df['acnt_id'] = cleaned['acnt_id'].values
#         norm_df['acnt_nm'] = cleaned['acnt_nm'].values
#         norm_df['influencer_scale_type'] = name

#         normalized_df_dict[name] = norm_df

#     normalized_all_df = pd.concat(normalized_df_dict.values(), ignore_index=True)
#     normalized_all_dic = normalized_all_df.to_dict(orient='index')

#     return normalized_all_df, normalized_all_dic


In [ ]:
# def normalize_influencer_scores(
#     influencer_scale_names, 
#     influencer_scale_df_list, 
#     reverse_columns=None, 
#     log_columns=None, 
#     feature_range=(0, 5)  ## 해당 부분 1,5점으로 변경
# ):
#     if reverse_columns is None:
#         reverse_columns = []
#     if log_columns is None:
#         log_columns = []

#     normalized_df_dict = {}

#     for name, df in zip(influencer_scale_names, influencer_scale_df_list):
#         cleaned = df.copy()

#         # 무한대 및 NaN 제거
#         float_cols = cleaned.select_dtypes(include='float64').columns
#         cleaned[float_cols] = cleaned[float_cols].replace([np.inf, -np.inf], np.nan)
#         cleaned = cleaned.dropna(subset=float_cols)

#         if cleaned.empty:
#             continue

#         norm_df = pd.DataFrame(index=cleaned.index)
#         for col in float_cols:
#             col_data = cleaned[col]
#             if col in log_columns:
#                 col_data = np.log1p(col_data)
#             scaler = MinMaxScaler(feature_range=feature_range)
#             norm_col = scaler.fit_transform(col_data.values.reshape(-1, 1))
#             if col in reverse_columns:
#                 norm_df[col] = feature_range[1] - norm_col.ravel()
#             else:
#                 norm_df[col] = norm_col.ravel()

#         norm_df['acnt_id'] = cleaned['acnt_id'].values
#         norm_df['acnt_nm'] = cleaned['acnt_nm'].values
#         norm_df['influencer_scale_type'] = name

#         normalized_df_dict[name] = norm_df

#     normalized_all_df = pd.concat(normalized_df_dict.values(), ignore_index=True)
#     normalized_all_dic = normalized_all_df.to_dict(orient='index')

#     return normalized_all_df, normalized_all_dic


In [ ]:
# from sklearn.preprocessing import MinMaxScaler

# def normalize_influencer_scores(
#     influencer_scale_names, 
#     influencer_scale_df_list, 
#     reverse_columns=None, 
#     log_columns=None, 
#     feature_range=(1, 5)
# ):
#     if reverse_columns is None:
#         reverse_columns = []
#     if log_columns is None:
#         log_columns = []

#     normalized_df_list = []

#     for name, df in zip(influencer_scale_names, influencer_scale_df_list):
#         cleaned = df.copy()

#         # 무한대 및 NaN 제거
#         float_cols = cleaned.select_dtypes(include='float64').columns
#         cleaned[float_cols] = cleaned[float_cols].replace([np.inf, -np.inf], np.nan)
#         cleaned = cleaned.dropna(subset=float_cols)

#         if cleaned.empty:
#             continue

#         # 관심 카테고리 첫 번째 값 추출
#         if 'interestcategory' in cleaned.columns:
#             cleaned['main_interest_category'] = cleaned['interestcategory'].apply(
#                 lambda x: str(x).split('@')[0] if pd.notnull(x) else '뷰티'
#             )
#         else:
#             cleaned['main_interest_category'] = '뷰티'

#         # 스케일 + 관심카테고리 조합별 정규화
#         grouped = cleaned.groupby('main_interest_category')

#         for category, group in grouped:
#             norm_df = pd.DataFrame(index=group.index)
#             for col in float_cols:
#                 col_data = group[col]
#                 if col in log_columns:
#                     col_data = np.log1p(col_data)
#                 scaler = MinMaxScaler(feature_range=feature_range)
#                 norm_col = scaler.fit_transform(col_data.values.reshape(-1, 1))
#                 if col in reverse_columns:
#                     norm_df[col] = feature_range[1] - norm_col.ravel()
#                 else:
#                     norm_df[col] = norm_col.ravel()

#             norm_df['acnt_id'] = group['acnt_id'].values
#             norm_df['acnt_nm'] = group['acnt_nm'].values
#             norm_df['influencer_scale_type'] = name
#             norm_df['main_interest_category'] = category

#             normalized_df_list.append(norm_df)

#     normalized_all_df = pd.concat(normalized_df_list, ignore_index=True)
#     normalized_all_dic = normalized_all_df.to_dict(orient='index')

#     return normalized_all_df, normalized_all_dic


In [64]:
nano = connected_flexmatch_score_table[connected_flexmatch_score_table['influencer_scale_type']=='nano']
micro = connected_flexmatch_score_table[connected_flexmatch_score_table['influencer_scale_type']=='micro']
mid = connected_flexmatch_score_table[connected_flexmatch_score_table['influencer_scale_type']=='mid']
macro = connected_flexmatch_score_table[connected_flexmatch_score_table['influencer_scale_type']=='macro']
mega = connected_flexmatch_score_table[connected_flexmatch_score_table['influencer_scale_type']=='mega']

In [65]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

def normalize_influencer_scores(
    influencer_scale_names, 
    influencer_scale_df_list, 
    reverse_columns=None, 
    log_columns=None, 
    feature_range=(1, 5)
):
    if reverse_columns is None:
        reverse_columns = []
    if log_columns is None:
        log_columns = []

    normalized_df_list = []

    for name, df in zip(influencer_scale_names, influencer_scale_df_list):
        cleaned = df.copy()

        # 무한대 및 NaN 제거
        float_cols = cleaned.select_dtypes(include='float64').columns
        cleaned[float_cols] = cleaned[float_cols].replace([np.inf, -np.inf], np.nan)
        cleaned = cleaned.dropna(subset=float_cols)

        if cleaned.empty:
            continue

        # 관심 카테고리 첫 번째 값 추출
        if 'interestcategory' in cleaned.columns:
            cleaned['main_interest_category'] = cleaned['interestcategory'].apply(
                lambda x: str(x).split('@')[0] if pd.notnull(x) else '뷰티'
            )
        else:
            cleaned['main_interest_category'] = '뷰티'

        # (관심카테고리, 스케일타입) 조합별 그룹화
        cleaned['influencer_scale_type'] = name  # 확실히 넣어두기
        grouped = cleaned.groupby(['main_category', 'influencer_scale_type'])

        for (category, scale_type), group in grouped:
            norm_df = pd.DataFrame(index=group.index)

            # 그룹 내 인원 수 체크
            if len(group) == 1:
                # 1명인 경우 모든 점수를 1.0으로 고정
                for col in float_cols:
                    norm_df.loc[group.index, col] = 3.0
            else:
                # 여러명인 경우 정규화 진행
                for col in float_cols:
                    col_data = group[col]
                    if col in log_columns:
                        col_data = np.log1p(col_data)

                    max_val = col_data.max()
                    min_val = col_data.min()

                    if max_val == min_val:
                        # 모든 값이 동일하면 1.0 고정
                        norm_df.loc[group.index, col] = 3.0
                    else:
                        scaler = MinMaxScaler(feature_range=feature_range)
                        norm_col = scaler.fit_transform(col_data.values.reshape(-1, 1))
                        if col in reverse_columns:
                            norm_df[col] = feature_range[1] - norm_col.ravel()
                        else:
                            norm_df[col] = norm_col.ravel()

            # 공통 컬럼 복사
            norm_df['acnt_id'] = group['acnt_id'].values
            norm_df['acnt_nm'] = group['acnt_nm'].values
            norm_df['influencer_scale_type'] = scale_type
            norm_df['main_interest_category'] = group['main_interest_category'].values
            norm_df['main_category'] = category
            norm_df['top_3_category'] = group['top_3_category'].values

            normalized_df_list.append(norm_df)

    normalized_all_df = pd.concat(normalized_df_list, ignore_index=True)
    normalized_all_dic = normalized_all_df.to_dict(orient='index')

    return normalized_all_df, normalized_all_dic


In [66]:
influencer_scale_names=['nano', 'micro', 'mid', 'macro', 'mega']
influencer_scale_df_list=[nano, micro, mid, macro, mega]

normalized_df, normalized_all_dic = normalize_influencer_scores(influencer_scale_names, influencer_scale_df_list, reverse_columns=None)


In [67]:
normalized_all_dic

{0: {'activity_score': 3.0,
  'follow_growth_rate': 3.0,
  'follower_retention_rate': 3.0,
  'avg_post_efficiency': 3.0,
  'avg_post_popularity': 3.0,
  'ad_efficiency': 3.0,
  'acnt_id': '17841450980480576',
  'acnt_nm': '__dongx2_',
  'influencer_scale_type': 'nano',
  'main_interest_category': '베이비/키즈',
  'main_category': '스포츠',
  'top_3_category': '스포츠@일상@다이어트/건강보조식품'},
 1: {'activity_score': 1.0,
  'follow_growth_rate': 1.0,
  'follower_retention_rate': 5.0,
  'avg_post_efficiency': 5.0,
  'avg_post_popularity': 1.0,
  'ad_efficiency': 3.0,
  'acnt_id': '17841400360358101',
  'acnt_nm': 'bro_bums',
  'influencer_scale_type': 'nano',
  'main_interest_category': '패션',
  'main_category': '일상',
  'top_3_category': '일상@스포츠@여행/관광'},
 2: {'activity_score': 5.0,
  'follow_growth_rate': 5.0,
  'follower_retention_rate': 1.0,
  'avg_post_efficiency': 1.0,
  'avg_post_popularity': 5.000000000000001,
  'ad_efficiency': 3.0,
  'acnt_id': '17841405735233010',
  'acnt_nm': 'flex_h.y',
  'influen

In [68]:
ssh.insert_query_with_lookup('op_mem_seller_score', list(normalized_all_dic.values()))

INSERT INTO op_mem_seller_score (activity_score, follow_growth_rate, follower_retention_rate, avg_post_efficiency, avg_post_popularity, ad_efficiency, acnt_id, acnt_nm, influencer_scale_type, main_interest_category, main_category, top_3_category, member_uid, user_id, is_connected) VALUES (%(activity_score)s, %(follow_growth_rate)s, %(follower_retention_rate)s, %(avg_post_efficiency)s, %(avg_post_popularity)s, %(ad_efficiency)s, %(acnt_id)s, %(acnt_nm)s, %(influencer_scale_type)s, %(main_interest_category)s, %(main_category)s, %(top_3_category)s, %(member_uid)s, %(user_id)s, %(is_connected)s)
inserted acnt_id: 17841450980480576
INSERT INTO op_mem_seller_score (activity_score, follow_growth_rate, follower_retention_rate, avg_post_efficiency, avg_post_popularity, ad_efficiency, acnt_id, acnt_nm, influencer_scale_type, main_interest_category, main_category, top_3_category, member_uid, user_id, is_connected) VALUES (%(activity_score)s, %(follow_growth_rate)s, %(follower_retention_rate)s, %(

In [69]:
# import os
# os.chdir("C:/Users/ehddl/Desktop/업무/code/Flexmatch_score/")

In [ ]:
# from modules.DB_connection_and_Load_conn_S3_data import *
# from modules.data_preprocessing import *
# from modules.connected_user_calcuate_flexmatch_score import *


# ssh = SSHMySQLConnector()
# ssh.load_config_from_json('C:/Users/ehddl/Desktop/업무/code/config/ssh_db_config.json') 
# ssh.connect()

# load_dotenv()
# aws_access_key = os.getenv("aws_accessKey")
# aws_secret_key = os.getenv("aws_secretKey")

# client = boto3.client(
#     's3',
#     aws_access_key_id=aws_access_key,
#     aws_secret_access_key=aws_secret_key,
#     region_name='ap-northeast-2'
# )

# def main():
#     # DB data loading
#     sales_info, seller_interest_info, conn_user_main_category_info = get_all_infos()

#     ## s3 data loading
#     bucket_name = 'flexmatch-data'
#     table_list = [
#         'CONN_v2_RECENT_USER_INFO_MTR',
#         'CONN_v2_TIME_SERIES_PROFILE_INFO',
#         'CONN_v2_BY_USER_ID_MEDIA_DTL_INFO',
#         'CONN_v2_BY_DATE_MEDIA_AGG_INFO',
#         'CONN_v2_PROFILE_INSIGHT_DTL',
#         'CONN_v2_MEDIA_INSIGHT_CUM',
#         'CONN_v2_PROFILE_INSIGHT_FOLLOWTYPE'
#     ]

#     # connected_user & not_connected_user common table
#     merged_data_by_table = load_last_weekly_instagram_data(bucket_name, table_list, target_filename='merged_data.parquet')
    
#     recent_user_info_mtr = merged_data_by_table['CONN_v2_RECENT_USER_INFO_MTR']['yesterday']
#     time_series_profile_info = merged_data_by_table['CONN_v2_TIME_SERIES_PROFILE_INFO']['yesterday']
#     by_user_id_media_dtl_info = merged_data_by_table['CONN_v2_BY_USER_ID_MEDIA_DTL_INFO']['yesterday']
#     by_date_media_agg_info = merged_data_by_table['CONN_v2_BY_DATE_MEDIA_AGG_INFO']['yesterday']

#     recent_user_info_mtr_2 = merged_data_by_table['CONN_v2_RECENT_USER_INFO_MTR']['today']
#     time_series_profile_info_2 = merged_data_by_table['CONN_v2_TIME_SERIES_PROFILE_INFO']['today']
#     by_user_id_media_dtl_info_2 = merged_data_by_table['CONN_v2_BY_USER_ID_MEDIA_DTL_INFO']['today']
#     by_date_media_agg_info_2 = merged_data_by_table['CONN_v2_BY_DATE_MEDIA_AGG_INFO']['today']

#     conn_profile_insight = merged_data_by_table['CONN_v2_PROFILE_INSIGHT_DTL']['yesterday']
#     conn_profile_insight_2 = merged_data_by_table['CONN_v2_PROFILE_INSIGHT_DTL']['today']

#     conn_media_insight = merged_data_by_table['CONN_v2_MEDIA_INSIGHT_CUM']['yesterday']
#     conn_media_insight_2 = merged_data_by_table['CONN_v2_MEDIA_INSIGHT_CUM']['today']

#     conn_profile_insight_followtype = merged_data_by_table['CONN_v2_PROFILE_INSIGHT_FOLLOWTYPE']['yesterday']
#     conn_profile_insight_followtype_2 = merged_data_by_table['CONN_v2_PROFILE_INSIGHT_FOLLOWTYPE']['today']

#     ## Data preprocessing
#     # ------- connected_user_data --------
#     c_unique_user = recent_user_info_mtr_2[recent_user_info_mtr_2['acnt_conn_yn']=='Y']['acnt_id'].to_list()
#     c_recent_user_info_mtr_2 = recent_user_info_mtr_2[recent_user_info_mtr_2['acnt_id'].isin(c_unique_user)]

#     c_time_series_profile_info = time_series_profile_info[time_series_profile_info['acnt_id'].isin(c_unique_user)]
#     c_time_series_profile_info_2 = time_series_profile_info_2[time_series_profile_info_2['acnt_id'].isin(c_unique_user)]

#     # by_user_id_media_dtl_info = by_user_id_media_dtl_info[by_user_id_media_dtl_info['acnt_id'].isin(c_unique_user)]
#     c_by_user_id_media_dtl_info_2 = by_user_id_media_dtl_info_2[by_user_id_media_dtl_info_2['acnt_id'].isin(c_unique_user)]

#     c_conn_profile_insight = conn_profile_insight[conn_profile_insight['acnt_id'].isin(c_unique_user)]
#     c_conn_profile_insight_2 = conn_profile_insight_2[conn_profile_insight_2['acnt_id'].isin(c_unique_user)]

#     c_conn_profile_insight_followtype = conn_profile_insight_followtype[conn_profile_insight_followtype['acnt_id'].isin(c_unique_user)]
#     c_conn_profile_insight_followtype_2 = conn_profile_insight_followtype_2[conn_profile_insight_followtype_2['acnt_id'].isin(c_unique_user)]

#     # connected_user 같은 경우에는 conn_media_insight 안에 있는 게 media_agg랑 동일하기 때문에 해당 부분을 쓰면 당장은 문제가 없음
#     unique_media = c_by_user_id_media_dtl_info_2['media_id'].unique()
#     # c_by_date_media_agg_info_2 = by_date_media_agg_info_2[by_date_media_agg_info_2['media_id'].isin(unique_media)]
#     c_conn_media_insight_2 = conn_media_insight_2[conn_media_insight_2['media_id'].isin(unique_media)]
        
#     user_info, timeseries, timeseries_2, user_followtype, user_followtype_2, media_info, media_agg, all_merged_df, media_engagement_merged_df, media_engagement_profile_merged_df, time_series_merged_df = conn_create_merged_df(
#                                                                                                                         c_recent_user_info_mtr_2,
#                                                                                                                         c_time_series_profile_info,
#                                                                                                                         c_time_series_profile_info_2,
#                                                                                                                         c_by_user_id_media_dtl_info_2,
#                                                                                                                         c_conn_media_insight_2,
#                                                                                                                         c_conn_profile_insight_followtype,
#                                                                                                                         c_conn_profile_insight_followtype_2)
    
#     user_info.loc[:,'influencer_scale_type'] = user_info.apply(influencer_scale_type, axis=1)

#     ## calculate flexmatch score - non_connected_user
#     activity_df = calculate_activity_score(media_info)
#     check_inf(activity_df)

#     growth_rate_df = calculate_follower_growth_rate(timeseries, timeseries_2)

#     # follower_engagment_df = calculate_follower_engagement(media_engagement_profile_merged_df)
#     # check_inf(follower_engagment_df)

#     follower_loyalty_df = calculate_follower_loyalty(conn_profile_insight_followtype_2, timeseries_2)
#     check_inf(follower_loyalty_df)

#     post_efficiency_df = calculate_post_efficiency_df(media_engagement_profile_merged_df)
#     check_inf(post_efficiency_df)
    
#     post_popularity_df = calculate_post_popularity_df(media_engagement_profile_merged_df)
#     check_inf(post_popularity_df)

#     # 광고효율성 계산할 때 필요
#     # db_merged_data = pd.merge(seller_interest_info, sales_info, on='add1')
    

#     ## create flexmatch score table by influencer scale type
#     connected_flexmatch_score_table = connected_user_flexmatch_score(user_info, activity_df, growth_rate_df, follower_loyalty_df, post_efficiency_df, post_popularity_df)
    
#     conn_user = seller_interest_info[(seller_interest_info['ig_user_id'].notnull()) & (seller_interest_info['ig_user_id'] != '')]
#     conn_user_interestcategory = conn_user[['ig_user_id', 'interestcategory']].rename({'ig_user_id' : 'acnt_id'}, axis=1)
    
#     # interest category 
#     connected_flexmatch_score_table = pd.merge(connected_flexmatch_score_table, conn_user_interestcategory, on='acnt_id', how='left')

#     connected_flexmatch_score_table['interestcategory'] = connected_flexmatch_score_table['interestcategory'].fillna('뷰티')
#     connected_flexmatch_score_table['interestcategory'] = connected_flexmatch_score_table['interestcategory'].apply(
#             lambda x: '뷰티' if pd.isna(x) or (isinstance(x, str) and x.strip() == '') else x)
    
#     category_map = {
#             'BABY/KIDS': '베이비/키즈',
#             'BEAUTY': '뷰티',
#             'FASHION': '패션',
#             'FOOD': '푸드',
#             'HEALTHY': '헬시',
#             'HOME/LIVING': '홈/리빙',
#             'SERVICE': '서비스',
#             'SPORT': '스포츠',
#             'TEST 카테고리.. TEST': '뷰티'
#         }

#     for k, v in category_map.items():
#         connected_flexmatch_score_table['interestcategory'] = connected_flexmatch_score_table['interestcategory'].str.replace(k, v)

#     # score table에 main category merge
#     conn_user_main_category_info = conn_user_main_category_info[['acnt_id', 'main_category', 'top_3_category']]
#     conn_user_main_category_info['acnt_id'] = conn_user_main_category_info['acnt_id'].astype(str)
#     connected_flexmatch_score_table = pd.merge(connected_flexmatch_score_table, conn_user_main_category_info, on='acnt_id', how='left')
    
#     # final preprocessing after table merge    
#     connected_flexmatch_score_table = connected_flexmatch_score_table.drop_duplicates(subset=['acnt_id', 'acnt_nm'])
    
#     nano = connected_flexmatch_score_table[connected_flexmatch_score_table['influencer_scale_type']=='nano']
#     micro = connected_flexmatch_score_table[connected_flexmatch_score_table['influencer_scale_type']=='micro']
#     mid = connected_flexmatch_score_table[connected_flexmatch_score_table['influencer_scale_type']=='mid']
#     macro = connected_flexmatch_score_table[connected_flexmatch_score_table['influencer_scale_type']=='macro']
#     mega = connected_flexmatch_score_table[connected_flexmatch_score_table['influencer_scale_type']=='mega']

#     # connected_user 추가

#     influencer_scale_names=['nano', 'micro', 'mid', 'macro', 'mega']
#     influencer_scale_df_list=[nano, micro, mid, macro, mega] # 여기에 connected user도 같이 포함하면 한번에 업로드 되지 않을까 함

#     normalized_df, normalized_all_dic = normalize_influencer_scores(influencer_scale_names, influencer_scale_df_list)
#     print(normalized_all_dic)

# if __name__=='__main__':
#     main()

2025-08-21 04:00:31,447| ERROR   | Socket exception: 현재 연결은 원격 호스트에 의해 강제로 끊겼습니다 (10054)


##### 크리에이터 광고 효율성 (IR용)

In [ ]:
revenue_dic = {
    'acnt_nm' : ['s_h_j_', 'siwolbubu_hyun', 'bong_camper83', 'binwoos', 'seojinii_', 'tingkerhee'],
    'sell_type' : ['flexmatch', 'other', 'flexmatch', 'flexmatch', 'other', 'other'],
    'total_revenue' : [6906000, 10937105, 7233100, 8759000, 7939664, 38449720],
    'total_order_cnt' : [84, 132, 122, 88, 89, 471]
    }

revenue_df = pd.DataFrame(revenue_dic)
revenue_merged_df = pd.merge(media_engagement_merged_df, revenue_df, on='acnt_nm', how='left')
revenue_merged_df = revenue_merged_df[['acnt_id', 'acnt_nm', 'follower_cnt', 'follow_cnt', 'media_cnt', 'sell_type', 'total_revenue', 'total_order_cnt', 'media_id', 'like_cnt', 'cmnt_cnt']]
revenue_merged_df

revenue_merged_df['post_efficiency'] = ((revenue_merged_df['like_cnt'] + revenue_merged_df['cmnt_cnt']) / revenue_merged_df['follower_cnt']) * 100
revenue_df_total = revenue_merged_df.groupby(['acnt_id', 'acnt_nm']).agg({
    'post_efficiency' : 'mean',
    'total_order_cnt' : 'first'
}).dropna()

revenue_df_total['advertisement_efficiency'] = (revenue_df_total['total_order_cnt'] / revenue_df_total['post_efficiency']) 
revenue_df_total.sort_values(by='advertisement_efficiency', ascending=False)

revenue_merged_df['engagement_per_post'] = ((revenue_merged_df['like_cnt'] + revenue_merged_df['cmnt_cnt']) / 25)
revenue_df_total = revenue_merged_df.groupby(['acnt_id', 'acnt_nm']).agg({
    'engagement_per_post' : 'mean',
    'total_revenue' : 'first',
    'total_order_cnt' : 'first'
}).dropna()

# 팔로워를 생각하지 않고, 25개의 콘텐츠 단위당 반응 비율을 계산
revenue_df_total['advertisement_efficiency'] = (revenue_df_total['total_order_cnt'] / revenue_df_total['engagement_per_post'] * 25) * 100
revenue_df_total.sort_values(by='advertisement_efficiency', ascending=False)

콘텐츠 효율성이 아닌 팔로워 참여도를 기준으로 계산했을 때

In [ ]:
engaged_df_2 = engaged_df.copy()
revenue_merged_df_2 = revenue_merged_df.groupby(['acnt_id', 'acnt_nm'])[['total_revenue', 'total_order_cnt']].first().dropna().reset_index()
revenue_df_total_2 = pd.merge(engaged_df_2, revenue_merged_df_2, on='acnt_id')

revenue_df_total_2['advertisement_efficiency'] = revenue_df_total_2['total_order_cnt'] / revenue_df_total_2['follower_total_engagement']
revenue_df_total_2[['acnt_id', 'acnt_nm', 'total_revenue', 'total_order_cnt', 'follower_total_engagement', 'advertisement_efficiency']].sort_values(by='advertisement_efficiency', ascending=False)

table merged

In [ ]:
# 크리에이터 활동성
creator_activity_score = activity_df[['acnt_id', 'avg_upload_interval']]

In [ ]:
# 트렌드지수
creator_trend_score = time_series_merged_df[['acnt_id', 'trend_score']]

In [ ]:
# 팔로워 참여도
follower_engagement = engaged_df[['acnt_id', 'follower_total_engagement']]

In [ ]:
# 팔로워 충성도
follower_loyalty = time_series_merged_df_copy[['acnt_id', 'follower_retention_rate']]

In [ ]:
# 콘텐츠 효율성
post_efficiency = post_efficiency_df.copy()

In [ ]:
# 광고효율성
revenue_df_total = revenue_df_total.reset_index()
advertisement_efficiency = revenue_df_total[['acnt_id', 'advertisement_efficiency']]

In [ ]:
df_list = [creator_activity_score, creator_trend_score, follower_engagement, follower_loyalty, post_efficiency, advertisement_efficiency]

In [ ]:
from functools import reduce

flexmatch_score = reduce(lambda left, right: pd.merge(left, right, on='acnt_id', how='left'), df_list)

In [ ]:
user_info_nm = user_info_2[['acnt_id', 'acnt_nm']]
flexmatch_score = pd.merge(flexmatch_score, user_info_nm, on='acnt_id')

In [ ]:
flexmatch_score = flexmatch_score[['acnt_id', 'acnt_nm', 'avg_upload_interval', 'trend_score', 'follower_total_engagement', 'follower_retention_rate', 'avg_post_efficiency', 'advertisement_efficiency']]
flexmatch_score.rename(columns={
    'avg_upload_interval' : '크리에이터 활동성',
    'trend_score' : '트렌드지수(팔로워순증가량)', 
    'follower_total_engagement' : '콘텐츠 참여도' ,
    'follower_retention_rate' : '팔로워 충성도',
    'avg_post_efficiency' : '콘텐츠 효율성',
    'advertisement_efficiency' : '광고 효율성'
}, inplace=True)

In [ ]:
flexmatch_score_2 = flexmatch_score.copy()
flexmatch_score_2.dropna(inplace=True)

In [ ]:
flexmatch_score_2

,acnt_id,acnt_nm,크리에이터 활동성,트렌드지수(팔로워순증가량),콘텐츠 참여도,팔로워 충성도,콘텐츠 효율성,광고 효율성
0,17841400361359004,s_h_j_,1.166667,0.057866,110.188904,100.00,0.045178,1859.316062
1,17841400561503844,binwoos,0.500000,-0.014956,626.642600,99.99,0.169730,518.470975
2,17841400591698216,tingkerhee,2.416667,-0.018509,2065.579146,99.98,0.483403,974.343202
4,17841401506106699,siwolbubu_hyun,0.958333,-0.075045,2094.996055,99.92,0.556736,237.096389
6,17841402936102997,seojinii_,2.875000,-0.002312,166.809026,100.00,0.240359,370.279723
10,17841453615191128,bong_camper83,1.500000,0.281793,2526.578680,100.00,2.506526,48.672935


In [ ]:
# from sklearn.preprocessing import MinMaxScaler

# columns = flexmatch_score_2.select_dtypes(include='float64').columns
# minmax = MinMaxScaler(feature_range=(0, 5))
# flexmatch_score_norm = minmax.fit_transform(flexmatch_score_2[columns])

# flexmatch_score_norm_df = pd.DataFrame(flexmatch_score_norm, columns=columns, index=flexmatch_score_2.index)
# flexmatch_score_norm_df['acnt_id'] = flexmatch_score_2['acnt_id']
# flexmatch_score_norm_df['acnt_nm'] = flexmatch_score_2['acnt_nm']

# flexmatch_score_norm_df['avg_upload_interval'] = 5 - flexmatch_score_norm_df['avg_upload_interval']
# flexmatch_score_norm_df


In [ ]:
from sklearn.preprocessing import MinMaxScaler

columns = flexmatch_score_2.select_dtypes(include='float64').columns
minmax = MinMaxScaler(feature_range=(0, 5))
flexmatch_score_norm = minmax.fit_transform(flexmatch_score_2[columns])

flexmatch_score_norm_df = pd.DataFrame(flexmatch_score_norm, columns=columns, index=flexmatch_score_2.index)
flexmatch_score_norm_df['acnt_id'] = flexmatch_score_2['acnt_id']
flexmatch_score_norm_df['acnt_nm'] = flexmatch_score_2['acnt_nm']

flexmatch_score_norm_df['크리에이터 활동성'] = 5 - flexmatch_score_norm_df['크리에이터 활동성']
flexmatch_score_norm_df

# 지금 이거 정규화 할 때는 그냥 이사람들을 전부 다 합쳐서 함께 정규화를 진행해서 그런데, 원래라면은 팔로워를 기준으로 뭐 메가 인플루언서, 마이크로 인플루언서 이런식으로 기준을 나눠서 정규화를 해야 조금 더 정확한 값이 될 것 같음.


,크리에이터 활동성,트렌드지수(팔로워순증가량),콘텐츠 참여도,팔로워 충성도,콘텐츠 효율성,광고 효율성,acnt_id,acnt_nm
0,3.596491,1.862339,0.000000,5.000,0.000000,5.000000,17841400361359004,s_h_j_
1,5.000000,0.841960,1.068647,4.375,0.253016,1.297324,17841400561503844,binwoos
2,0.964912,0.792182,4.046099,3.750,0.890213,2.556192,17841400591698216,tingkerhee
4,4.035088,0.000000,4.106968,0.000,1.039182,0.520322,17841401506106699,siwolbubu_hyun
6,0.000000,1.019133,0.117159,5.000,0.396492,0.888101,17841402936102997,seojinii_
10,2.894737,5.000000,5.000000,5.000,5.000000,0.000000,17841453615191128,bong_camper83


In [ ]:
# flexmatch_score_norm_df.to_csv("flexmatch_score_test.csv")